# * Data Science skills currently in demand *
<i>Author : Vinitha Palani</i><br>
<i>Date   : April 7, 2017<i><br>

## Table of contents <a id='Table-of-contents'></a>

1. [Problem Definition](#Problem)

2. [Required libraries](#Required-libraries)

3. [Load data](#Load-data)
    
4. [Checking the data](#Checking-the-data)
    - [Peek at your Data](#Peek)
    - [Dimensions](#Dimensions)
    - [Datatype of each feature](#Datatype)
    - [Descriptive Statistics](#Descriptive-stats)
    - [Class Distribution](#Class-dist)
    - [Correlation](#Correlation)
    - [Skew](#Skew)
    - [Takeaway](#Takeaway1)
    
5. [Missing data](#Missing-data)

6. [Exploratory Data analysis](#EDA)
    - [Univariate Analysis](#EDA)
    - [Takeaway](#Takeaway2)
    - [Multivariate Analysis](#Multivariate)
    - [Takeaway](#Takeaway3)
    - [Outliers](#Outliers)
    
7. [Feature Engineering](#Feature)
    - [Add new Features](#Feature)
    - [Remove Features](#Feature1)
    - [Apply Transformations](#Feature12)
    - [Convert catagorical variables](#Feature2)
    - [Standardize](#Feature3)
    - [Normalize](#Feature4)
    - [Make Binary](#Feature5)
    - [Scale](#Feature6)
    - [Bin](#Feature7)
    - [Correlation and Interaction variables](#Feature8)
    - [Testing](#Feature9)
    - [Repeat EDA](#Feature10)
    - [Takeaway](#Feature11)
  
8. [Feature Selection](#Feature-Selection)

9. [Algorithm Evaluation With Resampling Methods](#Resampling)

10. [Performance metrics](#metrics)

11. [Spot-check Algorithms](#Spot-check)

12. [Hyper-parameter Optimisation](#Hyperparameters)

13. [A single Pipeline](#Pipeline)

14. [Finalize the model](#Pickle)

15. [If I had more time..](#time)

16. [Acknowledgements](#Acknowledgement)



<a id='Problem'></a>

# Problem Definition

[[ go back to the top ]](#Table-of-contents)


#### What is the problem?
I would like to find out which Data Science skills are in demand in Atlanta, GA, as of April 2017.
Which combination of skills appear together? Is there a reason why they appear together, like is there a group of skills that are often required together for the financial sector(or Insurance or Retail) ? 
What are the job titles that are in vogue ?
Are more jobs offered FULL TIME (as opposed to PART-TIME or CONTRACT) ? 

###### Similar problems
Jesse Steinweg-Woods https://jessesw.com/Data-Science-Skills/

#### Why does the problem need to be solved?
This project is primarily undertaken for educational purposes.But I also hope to get some insights into how marketable my resume is given the current job market for data scientists in Atlanta, GA. 

#### How would I solve the problem?
The plan is to scrape the job postings on Dice.com with the title 'Data Scientist' for job-locations within 30 miles  radius of Atlanta(GA) using the python libraries urllib and BeautifulSoup.Then an exploratory analysis has to be done using Pandas and visualisation libraries such as Matplotlib and Seaborn.Clustering techniques are then to be used (using clustering algorithms in Scikit-learn) to find out if there are groups of skills("clusters") that tend to appear together in these job listings. 

#### Data Analysis Checklist 

<b>Did you specify the type of data analytic question (e.g. exploration, association causality) before touching the data?</b><br>
Yes, this is an exploratory problem<br>
<b>Did you define the metric for success before beginning?</b><br>
Although "success" cannot be quantified this case, the findings can be verified by comparing the results of the analysis with reports of the similar kind published by reliable sources.<br>
<b>Did you consider whether the question could be answered with the available data?</b><br>
I am currently scraping the data from just one job site(dice.com), other job sites like Indeed.com and Monster.com have restrictions placed by the site owners(as given in their respective robots.txt files). I am assuming that the postings on Dice.com are representative of all the current job openings for Data Scientists in Atlanta. Also my search is for the title 'Data Scientist' ,I am hoping the dice.com will bring up all the postings pertaining to Data Scientists even though the job-title is not precisely 'Data Scientist'.I also wanted to calculate the mean salary associated with each skill-set but most postings do not have any salary info.<br>


<a id='#Required-libraries'></a>

# Required Libraries

[[ go back to the top ]](#Table-of-contents)

In [1]:
# Python version
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))
# pandas
import pandas
print('pandas: {}'.format(pandas.__version__))
# scikit-learn
import sklearn
print('sklearn: {}'.format(sklearn.__version__))
import seaborn 
print('seaborn: {}'.format(seaborn.__version__))
import urllib
import bs4 #this is beautiful soup
%matplotlib inline

Python: 3.5.2 |Anaconda 4.2.0 (64-bit)| (default, Jul  5 2016, 11:41:13) [MSC v.1900 64 bit (AMD64)]
scipy: 0.18.1
numpy: 1.11.3
matplotlib: 1.5.3
pandas: 0.18.1
sklearn: 0.18.1
seaborn: 0.7.1


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

<a id='#Load-data'></a>

# Load data

[[ go back to the top ]](#Table-of-contents)



In [18]:
url = 'https://www.dice.com/jobs?q=Data+Scientist&l=Atlanta%2C+GA&searchid=6830934449416'
source = urllib.request.urlopen(url).read()
# parse html code
bs_tree = bs4.BeautifulSoup(source)
o = open("dicehtml.txt",'w', newline="\r\n")
for line in bs_tree:
    #print(line)
    o.write(str(line))
o.close()

C:\Users\Vini\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Vini\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [19]:
# total number of job listings
job_count_string = bs_tree.find(id = 'posiCountId').contents[0]
print(str(job_count_string))

1,078


In [20]:
# convert the string to a number (remove ',')
job_count = ''.join(n for n in str(job_count_string).split(','))
int(job_count)

1078

In [21]:
# 30 results per page, 
# so we need to scrape them page after page
num_pages = int(np.ceil(int(job_count)/30.0))
num_pages

36

In [23]:
import time
import socket
import requests
def get_tree(url):
    """
    Returns the BeautifulSoup parse tree 
    """
    # using 'requests' instead of urllib coz it takes care of bad url encoding
    try:
        source = requests.get(url)
        source.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        return None
    bs_tree = bs4.BeautifulSoup(source.text)
    return bs_tree

def get_job_urls(bs_tree):
    """
    Returns a list of job urls  
    """
    job_urls = []
    job_postings = []
    job_postings = bs_tree.findAll("ul")
    job_postings = [jp.find_all("a",{"class":"dice-btn-link loggedInVisited"}) for jp in job_postings if not jp.get('class') is None 
                  and ''.join(jp.get('class')) == "list-inline"] 
    job_postings = [jp for jp in job_postings if len(jp) is not 0]
    job_urls = [tag[0].get('href') for tag in job_postings]
    return job_urls 

def process_job_urls(job_urls):
    """
    Go to each job url and get the skill-set and employment-type
    """
    job_info = []
    for url in job_urls:
        print("Accessing  : {0} ".format(url))
        bs_tree = get_tree(url)
        if bs_tree is None: 
            print("Error accessing url : ",url)
            return None
        title = bs_tree.find_all("input",{"id":"estJTitle"})[0].get('value')
        skills = bs_tree.find_all("input",{"id":"estSkillText"})[0].get('value')
        emptype = bs_tree.find_all("input",{"id":"taxTermsTextId"})[0].get('value')
        job_info.append(str(title).replace(":",","))
        job_info.append(str(skills).replace(":",","))
        job_info.append(str(emptype).replace(":",","))
        time.sleep(2)
#        print("job_info",job_info)      
    return job_info

def write_csv(job_info):
    """
    Writes the job_info to a csv file  
    """
    for i in range(0,len(job_info),3):
        line = ':'.join(map(str, job_info[i:i+3])) 
        o.write(line)
        o.write("\r")
                   
o = open("job_info.csv",'a') 
o.write(":Title:Skill-set:Type")
o.write("\r")
#for page in range(num_pages):
#After a cursory glance at the search results though the search brings up 36 pages of hits not all of them are relevant 
#especially the ones towards the end ..hence limiting the pagination logic to the first 25 pages 
for page in range(25):
    if page > 1:
        url = "https://www.dice.com/jobs/q-Data_Scientist-l-Atlanta%2C_GA-radius-30-startPage-" + \
           str(page) + "-jobs?searchid=6830934449416"
        print("Accessing Page no.{0} : {1} ".format(page,url))           
        bs_tree = get_tree(url)
        if bs_tree is None:
            print("error fetching page no :",page)
            break;       
    job_urls = get_job_urls(bs_tree)
    job_info = process_job_urls(job_urls)
    write_csv(job_info)
    time.sleep(2)  
o.close()
print("Done!")

Accessing  : https://www.dice.com/jobs/detail/Sr.-Manager%2C-Data-Scientist-%26%2345-BHJOB2052_13161-Visionaire-Partners-Atlanta-GA-30339/10294321/BHJOB2052_13161?icid=sr1-1p&q=Data Scientist&l=Atlanta, GA 


C:\Users\Vini\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file C:\Users\Vini\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


Accessing  : https://www.dice.com/jobs/detail/Data-Scientist-Kforce-Inc.-Atlanta-GA-30339/kforcecx/ITEQG1605365?icid=sr2-1p&q=Data Scientist&l=Atlanta, GA 
Accessing  : https://www.dice.com/jobs/detail/Sr.-Manager%2C-Data-Scientist-%26%2345-Personalization-%26%2345-BHJOB2052_13158-Visionaire-Partners-Atlanta-GA-30339/10294321/BHJOB2052_13158?icid=sr3-1p&q=Data Scientist&l=Atlanta, GA 
Accessing  : https://www.dice.com/jobs/detail/Data-Scientist-Benvia-Atlanta-GA-30301/10119901/932391?icid=sr4-1p&q=Data Scientist&l=Atlanta, GA 
Accessing  : https://www.dice.com/jobs/detail/Data-Scientist-Themesoft-Inc-Atlanta-GA-30301/10123197/sushil_data?icid=sr5-1p&q=Data Scientist&l=Atlanta, GA 
Accessing  : https://www.dice.com/jobs/detail/Data-Scientist-Lorhan-Corporation-Atlanta-GA-30301/10440212/941119?icid=sr6-1p&q=Data Scientist&l=Atlanta, GA 
Accessing  : https://www.dice.com/jobs/detail/Data-Scientist-INSYS-Group-Alpharetta-GA-30004/insysus/17-00555?icid=sr7-1p&q=Data Scientist&l=Atlanta, GA 

In [8]:
#Loading to pandas
#Initially got this error 'utf-8' codec can't decode byte 0xa0 in position 1024: invalid start byte 
# tried encodings utf-8,utf-8-sig and finally cp1252 which worked.
j_cols = ['job_title', 'skill_set', 'employment_type']
job_info = pd.read_csv('job_info.csv',delimiter=':',names=j_cols,index_col=False,skiprows=1,encoding='cp1252')
job_info

job_title  \
0        Sr. Manager, Data Scientist - BHJOB2052_13161   
1                                       Data Scientist   
2    Sr. Manager, Data Scientist - Personalization ...   
3                                       Data Scientist   
4                                       Data Scientist   
5                                       Data Scientist   
6                                       Data Scientist   
7                                       Data Scientist   
8                                      Data Scientists   
9                                       Data Scientist   
10                                      Data Scientist   
11   AEM Consultant--- Blue Prism Architect---Data ...   
12       Data Scientist/Sr. Data Analyst (Tableau/SQL)   
13        Information security analyst /Data Scientist   
14                                 Data Scientist - GA   
15                           Data Engineer (Developer)   
16                          Data Analyst-Data Movement   
17                           Data Analyst-DBA and DBSA   
18   Workday HCM Data Conversion- Solution Engineer...   
19            Data Conversion- Solution Manager - USDC   
20                            Sr. Data Center Engineer   
21     Senior .NET Technical Lead/Manager - Data Focus   
22                               Sr Big Data Architect   
23                          Big Data Platform Engineer   
24                                         Data Mapper   
25                                   Big Data Engineer   
26        Oracle eBusiness Suite Product Data Hub Lead   
27          Data Architect      Atlanta, GA    Maddy J   
28                             Data Quality Specialist   
29                                        Data Analyst   
..                                                 ...   
720                               Senior iOS Developer   
721                                 Java Web Developer   
722                               Automation Test Lead   
723                   Senior Salesforce Technical Lead   
724              Sr. Corporate Quality Systems Manager   
725                  Oracle Fusion Reporting Developer   
726          Project Lead Kofax, Imaging/ Global scape   
727   Java Developer w/ MVC & PL/SQL - BHJOB2052_13137   
728            Cloud Software Engineer (JAVA - Hadoop)   
729                      Software Programmer  III-.Net   
730                                     .NET Developer   
731        Manager, Pricing Strategy - BHJOB2052_12896   
732                      Senior Manager of SAP Finance   
733                        Vmware System Engineer (05)   
734                                   Project Engineer   
735                                     Java Developer   
736                                   Project Engineer   
737             ServiceNow Developer - BHJOB2052_12920   
738            Windows Desktop/Server Support Engineer   
739                                    Program Manager   
740  Technical Product Owner / Manager - BHJOB2052_...   
741                             Ruby on Rails Engineer   
742                          UI Developer / Angular.JS   
743                          System Programmer Analyst   
744                                   Salesforce Admin   
745        IT Security Audit Manager - BHJOB2052_13125   
746                       Lead Cloud Software Engineer   
747                       Software Engineer I, II, III   
748                  Development Manager - Engineering   
749                                    ERP HCM Analyst   

                                             skill_set  \
0    statistics, data models, data mining, data ana...   
1    Analysis, Analytical, API, Data Mining, Java, ...   
2    statistics, data models, data mining, data ana...   
3         Python, Time Series, Cloud, Data Modeling, R   
4                                       Data Scientist   
5            Data Scientist,Python,Cloud,Data Modeling   
6                                        d

In [126]:
# Looks like a lot of postings has nothing to do with Data Science 
# Lets check the ones that are
ds_keywds = ['data','hadoop','tableau','python','spark','predictive','model','analytic','analyst',
                 'statistic','scientist','intelligence','sas','research','information','machine learning']
def find_ds_keywds(title):
    # checking the presence of some ds buzzwords in the job title..ofcourse anything with'data', we are interested 
    # courtesy : 'http://www.datasciencecentral.com/profiles/blogs/job-titles-for-data-scientists'
    return any(x in title.lower() for x in ds_keywds) 
    
job_info_ds = job_info[job_info['job_title'].apply(find_ds_keywds)]
job_info_ds

job_title  \
0        Sr. Manager, Data Scientist - BHJOB2052_13161   
1                                       Data Scientist   
2    Sr. Manager, Data Scientist - Personalization ...   
3                                       Data Scientist   
4                                       Data Scientist   
5                                       Data Scientist   
6                                       Data Scientist   
7                                       Data Scientist   
8                                      Data Scientists   
9                                       Data Scientist   
10                                      Data Scientist   
11   AEM Consultant--- Blue Prism Architect---Data ...   
12       Data Scientist/Sr. Data Analyst (Tableau/SQL)   
13        Information security analyst /Data Scientist   
14                                 Data Scientist - GA   
15                           Data Engineer (Developer)   
16                          Data Analyst-Data Movement   
17                           Data Analyst-DBA and DBSA   
18   Workday HCM Data Conversion- Solution Engineer...   
19            Data Conversion- Solution Manager - USDC   
20                            Sr. Data Center Engineer   
21     Senior .NET Technical Lead/Manager - Data Focus   
22                               Sr Big Data Architect   
23                          Big Data Platform Engineer   
24                                         Data Mapper   
25                                   Big Data Engineer   
26        Oracle eBusiness Suite Product Data Hub Lead   
27          Data Architect      Atlanta, GA    Maddy J   
28                             Data Quality Specialist   
29                                        Data Analyst   
..                                                 ...   
554              Senior HR Reporting Solutions Analyst   
555                        Operations Research Analyst   
556                       JD Edwards Developer Analyst   
558                                         QA Analyst   
559        Sr. Information Security Compliance Analyst   
563                       Information Security Analyst   
570                       Information Security Analyst   
574                     Hadoop Developer/Administrator   
579                           Tableau Server Architect   
582                          Sr Agile Business Analyst   
585                               DLP Security Analyst   
587                       Information Security Analyst   
592                                   Python Developer   
597                  Systems Analyst - with SQL skills   
601                       Senior Information Architect   
613            IT Database Administrator - Atlanta, GA   
622                   Business Analyst/Project Manager   
652                                  SAS Administrator   
667            Business Systems Analyst I-Applications   
669                  Director - Information Technology   
677                               ePMO Project Analyst   
689  Loss Mitigation and Workout Analyst, Merchant ...   
692                 SOC Security Analyst - SecureWorks   
694    Tier II Technical Support Analyst (MAC support)   
696             Lead Tier II Technical Support Analyst   
697                  Tier II Technical Support Analyst   
712                WMS Analyst (Professional Services)   
728            Cloud Software Engineer (JAVA - Hadoop)   
743                          System Programmer Analyst   
749                                    ERP HCM Analyst   

                                             skill_set  \
0    statistics, data models, data mining, data ana...   
1    Analysis, Analytical, API, Data Mining, Java, ...   
2    statistics, data models, data mining, data ana...   
3         Python, Time Series, Cloud, Data Modeling, R   
4                                       Data Scientist   
5            Data Scientist,Python,Cloud,Data Modeling   
6                                        d

In [127]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=2, stop_words='english')
# sparse representation of the counts using scipy.sparse.coo_matrix.
cvmatrix = cv.fit_transform(job_info_ds['skill_set'])
cv.vocabulary_

{'access': 0,
 'accounting': 1,
 'administration': 2,
 'adobe': 3,
 'aem': 4,
 'agile': 5,
 'ajax': 6,
 'ambari': 7,
 'analysis': 8,
 'analyst': 9,
 'analytical': 10,
 'analytics': 11,
 'apache': 12,
 'api': 13,
 'application': 14,
 'applications': 15,
 'arcgis': 16,
 'architect': 17,
 'architecture': 18,
 'architectures': 19,
 'arcsde': 20,
 'arima': 21,
 'asa': 22,
 'asp': 23,
 'automated': 24,
 'aws': 25,
 'azure': 26,
 'banking': 27,
 'based': 28,
 'bi': 29,
 'big': 30,
 'bigsql': 31,
 'blue': 32,
 'bteq': 33,
 'build': 34,
 'builder': 35,
 'building': 36,
 'business': 37,
 'capacity': 38,
 'case': 39,
 'cassandra': 40,
 'ccna': 41,
 'ceh': 42,
 'center': 43,
 'centric': 44,
 'cgi': 45,
 'change': 46,
 'cisco': 47,
 'cissp': 48,
 'civil': 49,
 'client': 50,
 'cloud': 51,
 'cms': 52,
 'code': 53,
 'collection': 54,
 'communication': 55,
 'compliance': 56,
 'computer': 57,
 'consultant': 58,
 'consulting': 59,
 'control': 60,
 'controls': 61,
 'crm': 62,
 'crystal': 63,
 'css': 64,
 

In [128]:
# 323 unique words are found , a matrix is created with one row each for 267 entries in job_info_ds (with 323 columns)
# every entry representing the frequency (of the word corresponding to that column) in the 
cvmatrix.todense().shape

(267, 324)

In [132]:
# adding up all the values in a given column will give the total number of times a word appears in the narrowed-down 323 joblistings  
dist = np.sum(cvmatrix.toarray(), axis=0)
word_count = {}
for tag, count in zip(cv.get_feature_names(), dist):
      word_count[tag] = count
sorted_cnts = pd.DataFrame(list(word_count.items()),columns=['Word','Count'])
sorted_cnts.sort_values(by=['Count'], ascending=False,inplace=True)
sorted_cnts.reset_index(drop=True)

Word  Count
0             data    150
1              sql     87
2         analysis     80
3          analyst     68
4       management     63
5          project     60
6         business     58
7           hadoop     51
8      development     43
9         security     37
10        database     32
11            java     32
12           excel     31
13          python     31
14             big     30
15         manager     29
16    architecture     29
17          oracle     28
18        modeling     28
19       architect     24
20         testing     23
21           linux     22
22           cloud     22
23           agile     21
24       developer     17
25    intelligence     17
26        research     16
27              bi     16
28          server     16
29           spark     15
..             ...    ...
294           ccna      2
295            ssl      2
296          point      2
297       firewall      2
298            pre      2
299          refer      2
300     programmer      2
301       spotfire      2
302            asa      2
303      mapreduce      2
304           ssas      2
305       planning      2
306         jquery      2
307      publisher      2
308         google      2
309           flow      2
310         vmware      2
311           knox      2
312           blue      2
313             hr      2
314           olap      2
315            ups      2
316  manufacturing      2
317            emc      2
318       protocol      2
319         threat      2
320            aem      2
321     production      2
322         client      2
323    statistical      2

[324 rows x 2 columns]

In [151]:
sorted_cnts[:50]

Word  Count
209          data    150
214           sql     87
169      analysis     80
171       analyst     68
105    management     63
104       project     60
193      business     58
300        hadoop     51
22    development     43
65       security     37
123      database     32
21           java     32
68          excel     31
2          python     31
265           big     30
160       manager     29
215  architecture     29
275        oracle     28
260      modeling     28
101     architect     24
127       testing     23
239         linux     22
167         cloud     22
66          agile     21
115     developer     17
233  intelligence     17
183      research     16
97             bi     16
31         server     16
7           spark     15
306  requirements     15
280    analytical     14
8           mysql     13
76        tableau     13
184          hive     13
266     lifecycle     13
319    powerpoint     13
208    consulting     12
288       windows     12
162    experience     12
33            etl     12
230          unix     11
270        skills     11
222     microsoft     11
164         hbase     11
136            ms     11
181        mining     10
264       metrics      9
124        degree      9
43         access      9

In [152]:
# whoa! 'R' a no-show! how come ? also d3.js  ..trying min_df = 1
cv1 = CountVectorizer(min_df=1, stop_words='english')
# sparse representation of the counts using scipy.sparse.coo_matrix.
cvmatrix1 = cv1.fit_transform(job_info_ds['skill_set'])
cv1.vocabulary_

{'10': 0,
 '11g': 1,
 '12c': 2,
 '256': 3,
 '365': 4,
 '3yrs': 5,
 '837': 6,
 'ab': 7,
 'abinitio': 8,
 'access': 9,
 'accounting': 10,
 'ace': 11,
 'aci': 12,
 'activities': 13,
 'administration': 14,
 'administrator': 15,
 'adobe': 16,
 'adoop': 17,
 'aem': 18,
 'agile': 19,
 'ai': 20,
 'airline': 21,
 'aix': 22,
 'ajax': 23,
 'alpharetta': 24,
 'alteryx': 25,
 'alto': 26,
 'amazon': 27,
 'ambari': 28,
 'aml': 29,
 'analysis': 30,
 'analyst': 31,
 'analytical': 32,
 'analytics': 33,
 'apache': 34,
 'apex': 35,
 'api': 36,
 'app': 37,
 'application': 38,
 'applications': 39,
 'appropriate': 40,
 'arcgis': 41,
 'architect': 42,
 'architecture': 43,
 'architectures': 44,
 'arcsde': 45,
 'arcsight': 46,
 'arima': 47,
 'artificial': 48,
 'asa': 49,
 'asp': 50,
 'atscale': 51,
 'automated': 52,
 'automation': 53,
 'aws': 54,
 'azure': 55,
 'ba': 56,
 'banking': 57,
 'based': 58,
 'bash': 59,
 'basic': 60,
 'bi': 61,
 'big': 62,
 'biginsights': 63,
 'bigsql': 64,
 'blue': 65,
 'bo': 66,
 'b

In [166]:
job_info_ds[job_info_ds['skill_set'].apply(lambda skill : 'd3' in skill)]                             

Empty DataFrame
Columns: [job_title, skill_set, employment_type]
Index: []

In [168]:
job_info_ds[job_info_ds['skill_set'].apply(lambda skill : ' R' in skill or 'R 'in skill)]                             

job_title  \
3                                       Data Scientist   
14                                 Data Scientist - GA   
17                           Data Analyst-DBA and DBSA   
23                          Big Data Platform Engineer   
24                                         Data Mapper   
33                                      Data Scientist   
44                                 Data Scientist - GA   
47                           Data Analyst-DBA and DBSA   
53                          Big Data Platform Engineer   
54                                         Data Mapper   
66                       Solution Architect (Big Data)   
67                           Enterprise Data Architect   
71                                  Big Data Architect   
72                       Data Center Network Architect   
74                                      Data Architect   
81            Linux System Data Center Design Engineer   
91                                Revenue Data Analyst   
96                                  Big Data Architect   
102                             Data Reporting Analyst   
105      Data Analyst Lead - Monitoring Tools Resource   
116                      Data Center Network Architect   
117                      Data Center Network Architect   
123                Data Analyst/Reports Developer (05)   
126  Financial Data Reporting Analyst // 6 Month Co...   
131        Computer Operator (Data Center) - 3rd Shift   
132                       Data Center Architect for GA   
135                               Senior UX Researcher   
138                   Senior Project/Marketing Analyst   
139                                       Statistician   
141                         Security Analytics Manager   
..                                                 ...   
152    Business Analyst - Software - Actimize division   
156                                   Business Analyst   
160                             BI Analytics Developer   
163                          Digital Analytics Manager   
166                                    Hadoop Engineer   
167                                   Business Analyst   
205                          Advanced Business Analyst   
213                         Technical Business Analyst   
225                       Information Security Analyst   
250                                L3 Business Analyst   
261              Esri ArcSDE Geodatabase Administrator   
269  Application Developer./Business Intelligence D...   
292                      Information Security Engineer   
298              Technical Application Support Analyst   
300                        Tableau, BO, HANA Reporting   
334                                   Hadoop Architect   
342                      Business Analyst - Salesforce   
348                    Business Intelligence Developer   
371                  Senior Technical Business Analyst   
424                 Senior Salesforce Business Analyst   
425                 Senior Salesforce Business Analyst   
463              Operational Policy Research Associate   
470                              Lead Business Analyst   
508                         Security Analyst (MID SOC)   
512                         Technical Business Analyst   
517                         Business / Reports Analyst   
574                     Hadoop Developer/Administrator   
669                  Director - Information Technology   
689  Loss Mitigation and Workout Analyst, Merchant ...   
743                          System Programmer Analyst   

                                             skill_set  \
3         Python, Time Series, Cloud, Data Modeling, R   
14        Data Science, Marketing Analytics, Hadoop, R   
17   Agile, Analysis, Analyst, Database, Database A...   
23   Agile, Analysis, Architecture, Business Requir...   
24   Analytical Skills, Business Requirements, CGI,...   
33        Python, Time Series, Cloud, Data Modeling, R   
44        Data Science, Marketing Analytic

In [156]:
print(cv)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=2,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)


In [191]:
# note that CountVectorizer discards "words" that contain only one character, such as "R"
# CountVectorizer also transforms all words into lowercase
cv1 = CountVectorizer(min_df=2, stop_words='english')
tokenize = cv1.build_tokenizer()
tokenize("Python, Time Series, Cloud, Data Modeling, R")


['Python', 'Time', 'Series', 'Cloud', 'Data', 'Modeling']

In [212]:
# many programming languages are just one character long , 'R' , 'C' 
# tweaking the 'token_pattern' parameter to include 'R' or 'C'
# Testing the new pattern
cv1 = CountVectorizer(min_df=1, stop_words='english', token_pattern=r'(?u)\b\w\w+\b|r|c' ,tokenizer=None)
tokenize = cv1.build_tokenizer()
tokenize("Python, Time Series, Cloud, R ,Data Modeling")

['Python', 'Time', 'Series', 'Cloud', 'Data', 'Modeling']

In [213]:
#cvmatrix1 = cv1.fit_transform(job_info_ds['skill_set'])
cvmatrix1 = cv1.fit_transform(["Python, Time Series, Cloud, Data Modeling, R"])
cv1.vocabulary_

{'cloud': 0,
 'data': 1,
 'modeling': 2,
 'python': 3,
 'r': 4,
 'series': 5,
 'time': 6}

In [221]:
R_job_ds = job_info_ds[job_info_ds['skill_set'].apply(lambda skill : ' R' in skill or 'R 'in skill)] 

In [228]:
# correcting the token_pattern 
#cv1 = CountVectorizer(min_df=2, stop_words='english', token_pattern=r'(?u)\b\w\w+\b|r|c' ,tokenizer=None)
cv1 = CountVectorizer(min_df=1, stop_words='english', token_pattern=r'(?u)\b\w\w+\b|[r]$' ,tokenizer=None)
#cvmatrix1 = cv1.fit_transform(job_info_ds['skill_set'])
#cvmatrix1 = cv1.fit_transform(["Python, Time Series, Cloud, Data Modeling, R","OBIEE ,PL/SQL , BI Publisher Reports","DC Architecture, Nexus, R&S and WAN networking"])
cvmatrix1 = cv1.fit_transform(R_job_ds['skill_set'])
dist = np.sum(cvmatrix1.toarray(), axis=0)
word_count = {}
for tag, count in zip(cv1.get_feature_names(), dist):
      word_count[tag] = count
sorted_cnts1 = pd.DataFrame(list(word_count.items()),columns=['Word','Count'])
sorted_cnts1.sort_values(by=['Count'], ascending=False,inplace=True)
sorted_cnts1.reset_index(drop=True)
sorted_cnts1[sorted_cnts1['Word'] == 'r']

Word  Count
99    r      4

In [210]:
job_info_ds[job_info_ds['skill_set'].apply(lambda skill : ' R' in skill or 'R 'in skill)]   

job_title  \
3                                       Data Scientist   
14                                 Data Scientist - GA   
17                           Data Analyst-DBA and DBSA   
23                          Big Data Platform Engineer   
24                                         Data Mapper   
33                                      Data Scientist   
44                                 Data Scientist - GA   
47                           Data Analyst-DBA and DBSA   
53                          Big Data Platform Engineer   
54                                         Data Mapper   
66                       Solution Architect (Big Data)   
67                           Enterprise Data Architect   
71                                  Big Data Architect   
72                       Data Center Network Architect   
74                                      Data Architect   
81            Linux System Data Center Design Engineer   
91                                Revenue Data Analyst   
96                                  Big Data Architect   
102                             Data Reporting Analyst   
105      Data Analyst Lead - Monitoring Tools Resource   
116                      Data Center Network Architect   
117                      Data Center Network Architect   
123                Data Analyst/Reports Developer (05)   
126  Financial Data Reporting Analyst // 6 Month Co...   
131        Computer Operator (Data Center) - 3rd Shift   
132                       Data Center Architect for GA   
135                               Senior UX Researcher   
138                   Senior Project/Marketing Analyst   
139                                       Statistician   
141                         Security Analytics Manager   
..                                                 ...   
152    Business Analyst - Software - Actimize division   
156                                   Business Analyst   
160                             BI Analytics Developer   
163                          Digital Analytics Manager   
166                                    Hadoop Engineer   
167                                   Business Analyst   
205                          Advanced Business Analyst   
213                         Technical Business Analyst   
225                       Information Security Analyst   
250                                L3 Business Analyst   
261              Esri ArcSDE Geodatabase Administrator   
269  Application Developer./Business Intelligence D...   
292                      Information Security Engineer   
298              Technical Application Support Analyst   
300                        Tableau, BO, HANA Reporting   
334                                   Hadoop Architect   
342                      Business Analyst - Salesforce   
348                    Business Intelligence Developer   
371                  Senior Technical Business Analyst   
424                 Senior Salesforce Business Analyst   
425                 Senior Salesforce Business Analyst   
463              Operational Policy Research Associate   
470                              Lead Business Analyst   
508                         Security Analyst (MID SOC)   
512                         Technical Business Analyst   
517                         Business / Reports Analyst   
574                     Hadoop Developer/Administrator   
669                  Director - Information Technology   
689  Loss Mitigation and Workout Analyst, Merchant ...   
743                          System Programmer Analyst   

                                             skill_set  \
3         Python, Time Series, Cloud, Data Modeling, R   
14        Data Science, Marketing Analytics, Hadoop, R   
17   Agile, Analysis, Analyst, Database, Database A...   
23   Agile, Analysis, Architecture, Business Requir...   
24   Analytical Skills, Business Requirements, CGI,...   
33        Python, Time Series, Cloud, Data Modeling, R   
44        Data Science, Marketing Analytic

That looks about right! 

In [237]:
# looks like we need to capture 2-word phrases like big data , time series etc , hence setting ngram_range = (1,2) 
cv1 = CountVectorizer(min_df=2, stop_words='english', token_pattern=r'(?u)\b\w\w+\b|[r]$' ,tokenizer=None, ngram_range=(1, 2))
cvmatrix1 = cv1.fit_transform(job_info_ds['skill_set'])
dist = np.sum(cvmatrix1.toarray(), axis=0)
word_count = {}
for tag, count in zip(cv1.get_feature_names(), dist):
      word_count[tag] = count
sorted_cnts1 = pd.DataFrame(list(word_count.items()),columns=['Word','Count'])
sorted_cnts1.sort_values(by=['Count'], ascending=False,inplace=True)
sorted_cnts1.head(50)

Word  Count
283                   data    150
500                    sql     87
98                analysis     80
99                 analyst     68
232             management     63
487                project     60
277               business     58
167                 hadoop     51
194            development     43
211               security     37
236               database     32
193                   java     32
185                 python     31
212                  excel     31
228                    big     30
10            architecture     29
445                manager     29
163               big data     29
436                 oracle     28
151       analysis analyst     28
398               modeling     28
231              architect     24
71                 testing     23
136                  linux     22
450                  cloud     22
379                  agile     21
560        project project     18
356          data analysis     18
413              developer     17
679           intelligence     17
169     project management     16
322                     bi     16
495                 server     16
676       business analyst     16
636               research     16
687     manager management     15
171           requirements     15
187                  spark     15
287          data modeling     14
596             analytical     14
407  business intelligence     14
590         agile analysis     14
220                tableau     13
595              lifecycle     13
420                  mysql     13
106                   hive     13
180             powerpoint     13
364                    etl     12
447             experience     12
637       analyst business     12

In [345]:
#Excluding some words/phrases from the first 50 entries to narrow the list further down to the names of technologies/tools
my_additional_stop_words = ['analysis', 'analyst', 'management', 'project', 'development', 
                'database', 'manager', 'analysis analyst', 'testing', 'project project', 
                'data analysis', 'developer', 'intelligence', 'project management', 'server', 'manager management', 
                 'requirements', 'analytical', 'experience', 'analyst business', 'security', 
                 'research', 'lifecycle', 'windows', 'consulting', 'skills',
                'programming', 'supervision', 'degree', 'center', 'sql sql', 'engineer', 'analytics',
                'access', 'process', 'office', 'validation', 'information', 'tools', 'quality', 'administration',
                 'application', 'infrastructure', 'application', 'scripts', 'http', 'video', 'enterprise', 
                'net', 'design', 'networks', 'hardware', 'ms', 'sql sql']
from sklearn.feature_extraction import text 
cv1 = CountVectorizer(min_df=2, stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words), token_pattern=r'(?u)\b\w\w+\b|[r]$' ,
                      tokenizer=None, ngram_range=(1, 2))
cvmatrix1 = cv1.fit_transform(job_info_ds['skill_set'])
dist = np.sum(cvmatrix1.toarray(), axis=0)
word_count = {}
for tag, count in zip(cv1.get_feature_names(), dist):
      word_count[tag] = count
sorted_cnts1 = pd.DataFrame(list(word_count.items()),columns=['Word','Count'])
sorted_cnts1.sort_values(by=['Count'], ascending=False,inplace=True)
# exclude these words unless they occur along with other words
drop_list = ['data', 'big', 'architect', 'microsoft', 'mining', 'metrics', 'business', 'modeling', 'sales', 'operations']
sorted_cnts1 = sorted_cnts1[sorted_cnts1['Word'].apply(lambda word : False if (any(x in word for x in drop_list) & len(word.split()) == 1) else True)]
sorted_cnts1[sorted_cnts1["Word"] != 'sql sql']

Word  Count
500                    sql     87
260                 hadoop     51
299                   java     32
342                  excel     31
284                 python     31
257               big data     29
125                 oracle     28
212                  linux     22
137                  cloud     22
58                   agile     21
81                      bi     16
287                  spark     15
458          data modeling     14
156                   hive     13
106                  mysql     13
277             powerpoint     13
354                tableau     13
28                     etl     12
165         data architect     12
135                  hbase     11
95                    unix     11
436            data mining     10
222                    aws      9
225                    sas      8
428                    sap      8
460     agile architecture      8
490                  cisco      8
441              scripting      8
302  architecture business      8
264                  kafka      8
..                     ...    ...
185         sql subversion      2
183             scrum sdlc      2
178       programmer scrum      2
175          usage logging      2
174        salesforce sfdc      2
216              cissp ceh      2
217        facing business      2
218        hadoop hcatalog      2
243     logging monitoring      2
259                 impala      2
258         visualizations      2
256   teradata informatica      2
254              data java      2
251              marketing      2
249           sql teradata      2
247      javascript jquery      2
245              pig oozie      2
244                    emc      2
242            sql telecom      2
220           python arima      2
240       architecture edi      2
239             python use      2
234               planning      2
231        centric network      2
230       architecture big      2
229                 spring      2
227             subversion      2
224           learning rpa      2
221        mapreduce spark      2
546                    ceh      2

[530 rows x 2 columns]

In [349]:
job_info_ds[job_info_ds['skill_set'].apply(str.lower).apply(lambda skill : 'bi' in skill)]

job_title                                          skill_set                                    employment_type
11   AEM Consultant--- Blue Prism Architect---Data ...  AEM Consultant--- Blue Prism Architect---Data ...  Full Time, Contract Corp-To-Corp, Contract Ind...
22                               Sr Big Data Architect                                 Big Data Architect                                          Full Time
41   AEM Consultant--- Blue Prism Architect---Data ...  AEM Consultant--- Blue Prism Architect---Data ...  Full Time, Contract Corp-To-Corp, Contract Ind...
52                               Sr Big Data Architect                                 Big Data Architect                                          Full Time
61                             Data Solutions Engineer  Analytics, Data Solutions, MuleSoft, AWS, Wave...                                          Full Time
66                       Solution Architect (Big Data)  Solution Architecture, Big Data, RDBMS (Data W...                                          Full Time
68                    Senior Enterprise Data Architect  Senior Enterprise Data Architect, Data, ETL, B...                                          Full Time
69                  Cloud/Big Data Technical Architect                            Big Data, Cloud, Google                                          Full Time
70           Google Cloud/Big Data Technical Architect                 BI/Big Data, Google Cloud Platform                                          Full Time
71                                  Big Data Architect  Big Data, Hadoop, Oracle, SQL, MySQL, Pentaho,...                                          Full Time
86                              Data Security Engineer  data etl big data bi php python sql shell tani...                                             C2H W2
90                      Big Data Application Architect  Big Data,spera, PowerBI, Tableau, SPSS, Spotfi...                 Contract Corp-To-Corp, Contract W2
92                  Machine Learning / Big Data Expert  Machine Learning, Big Data, Java, Scala, Pytho...  Full Time, Contract Corp-To-Corp, Contract Ind...
93                                      Data Architect               BIG DATA, ETL, ORACLE, LINUX, HADOOP                                                NaN
94                                  Big Data Architect  Big Data Architecture, Big Data Implementation...                 Full Time, C2H Independent, C2H W2
96                                  Big Data Architect  Big Data/Hadoop Architecture, IBM BigInsights ...                              Contract Corp-To-Corp
97                                  Big Data Architect                                 Big Data Architect  Full Time, Contract Corp-To-Corp, Contract Ind...
99                                  Big Data Architect  Big Data/Hadoop Architecture, HDFS, MR2, Hive,...  Full Time, Contract Corp-To-Corp, Contract Ind...
100                    Big Data/Java Product Architect  Java, Spring, Cloud, AWS, Big Data, MongoDB, N...                     Full Time, Contract W2, C2H W2
110                                 Big Data Architect                            HADOOP, BI ARCHITECTURE                                        Contract W2
113                     Data Specialist(ETL.Ab Initio)  AbInitio development experience,generic graphs...  Contract Corp-To-Corp, Contract Independent, C...
114                                 Big Data Architect  HDFS, MR2, Hive, HBase, Sqoop, Oozie, BigSQL, ...  Full Time, Contract Corp-To-Corp, Contract W2,...
124                  Big Data Architect/Developer role  Big Data Architect Developer Spark Kafka Cassa...  Contract Corp-To-Corp, Contract Independent, C...
127            Big Data Developer- Immediate Interview  Big Data Developer, Spark, Kafka, data sources...  Full Time, Contract Corp-To-Corp, Contract Ind...
128                  Application Support/ Data Analyst  Analyst, BI Tools, Change Management, Developm...  Contract Inde

In [348]:
job_info_ds[job_info_ds['skill_set'].apply(str.lower).apply(lambda skill : 'oracle' in skill)]

job_title                                          skill_set                                    employment_type
18   Workday HCM Data Conversion- Solution Engineer...  Analysis, Consulting, Development, ERP, Excel,...                                          Full Time
19            Data Conversion- Solution Manager - USDC  Analysis, Consulting, Development, Excel, HTTP...                                          Full Time
26        Oracle eBusiness Suite Product Data Hub Lead                      Oracle EBS, MDM, Data Quality  Contract Corp-To-Corp, C2H Corp-To-Corp, Contr...
48   Workday HCM Data Conversion- Solution Engineer...  Analysis, Consulting, Development, ERP, Excel,...                                          Full Time
49            Data Conversion- Solution Manager - USDC  Analysis, Consulting, Development, Excel, HTTP...                                          Full Time
56        Oracle eBusiness Suite Product Data Hub Lead                      Oracle EBS, MDM, Data Quality  Contract Corp-To-Corp, C2H Corp-To-Corp, Contr...
60                                      Data Architect  Architect, Architecture, Business Intelligence...                                          Full Time
71                                  Big Data Architect  Big Data, Hadoop, Oracle, SQL, MySQL, Pentaho,...                                          Full Time
74                                      Data Architect  Architecture, Business Requirements, CTO, Deve...                                          Full Time
88                                      Data Architect  Architecture, Business Intelligence, Business ...  Full Time, Contract Corp-To-Corp, Contract Ind...
93                                      Data Architect               BIG DATA, ETL, ORACLE, LINUX, HADOOP                                                NaN
102                             Data Reporting Analyst  MS Excel, Power Query, Charting Tools, SharePo...  Contract Corp-To-Corp, Contract Independent, C...
111                                       Data Analyst  Analyst, Database, ERP, Excel, Oracle, SQL, SQ...                                        Contract W2
122  Oracle DBA w/ Data Migration and Systems Exper...  Oracle DBA w/ Data Migration and Systems Exper...                                          Full Time
134  Data Architect / ETL Architect / Teradata Arch...  Data Architect, Data Modeling, Teradata, Infor...                                        Contract W2
139                                       Statistician  Analysis, Analytical Skills, Capacity Planning...                                          Full Time
142                             Database Administrator  Oracle Database 11g / 12c system internals, un...                                          Full Time
149                                 Operations Analyst  Analysis, Analyst, Analytical Skills, Apache, ...                                          Full Time
203                    Business Intelligence Developer  Data warehousing / Business Intelligence exper...  Full Time, Contract Corp-To-Corp, Contract Ind...
235                Sr. Hadoop Developer/Architect (05)  Hadoop, Big Data, Java, Business Intelligence,...                                          Full Time
236       BI Analyst w/ Fortune 500 Company in Atlanta  Analysis, Analyst, Architecture, BI Tools, Bus...                                          Full Time
250                                L3 Business Analyst  Analysis, Analyst, Architecture, Business Anal...                                          Full Time
298              Technical Application Support Analyst  Analysis, Analyst, Cisco, CMS, Consulting, Cus...                                          Full Time
317                    Business Intelligence Developer                        BI, OLAP, SQL, Oracle, Qlik                                        Contract W2
334                                   Hadoop Architect  Architecture, Business Requirements, Developme...               

<a id='#Checking-the-data'></a>

# Checking the data

[[ go back to the top ]](#Table-of-contents)
The next step is to look at the data we're working with. Even curated data sets from the government can have errors in them, and it's vital that we spot these errors before investing too much time in our analysis.<br>
Generally, we're looking to answer the following questions:<br>
Is there anything wrong with the data?<br>
Are there any quirks with the data?<br>
Do I need to fix or remove any of the data?<br>
<br>
For small datasets visualize using a pairplot to check for weirdness in the data , get the overall picture check if the data is linearly separable.<br>
example : sb.pairplot(iris_data.dropna(), hue='class')<br>
For larger datasets with lots of dimensions use Dimentionality Reduction techniques to visualize after variable conversion<br>
<b>PCA</b><br>
<b>t-SNE</b>

<a id='#Peek'></a>

### Step 1 : Peek at your data 

[[ go back to the top ]](#Table-of-contents)
Use head()

<a id='#Dimensions'></a>

### Step 2 : Dimensions

[[ go back to the top ]](#Table-of-contents)
Use df.shape

<a id='#Datatype'></a>

### Step 3 : Datatype of each feature

[[ go back to the top ]](#Table-of-contents)
Use info()

<a id='#Descriptive-stats'></a>

### Step 4 : Descriptive Statistics

[[ go back to the top ]](#Table-of-contents)
Use describe() & describe(include=[‘O’])

<a id='#Class-dist'></a>

### Step 5 : Class Distribution

[[ go back to the top ]](#Table-of-contents)
class_counts = df.groupby('class').size() <br>
Classification problems with imbalance classes require special treatment<br>

<a id='#Correlation'></a>

### Step 6 : Correlation

[[ go back to the top ]](#Table-of-contents)
correlations = df.corr(method='pearson')<br>
If you have ordinal data, you will want to use Spearman's rank-order correlation or a Kendall's Tau Correlation instead of the Pearson product-moment correlation. 

<a id='#Skew'></a>

### Step 7 : Skew

[[ go back to the top ]](#Table-of-contents)
skew = df.skew()

<a id='#Takeaway1'></a>

### Takeaway from "Checking the data"
What did I learn? <br>
[[ go back to the top ]](#Table-of-contents)
##### What is the distribution of numerical feature values across the samples?
##### What is the distribution of categorical feature values across the samples?
##### Which features are missing data?
##### Can anomalies/outliers/anything weird be spotted?
### Assumtions based on data analysis
We arrive at following assumptions based on data analysis done so far. We may validate these assumptions further before taking appropriate actions.
###### Correlating.
We want to know how well does each feature correlate with Survival. We want to do this early in our project and match these quick correlations with modelled correlations later in the project.
###### Completing.
We may want to complete Age feature as it is definitely correlated to survival.<br>
We may want to complete the Embarked feature as it may also correlate with survival or another important feature.
###### Correcting.
Ticket feature may be dropped from our analysis as it contains high ratio of duplicates (22%) and there may not be a correlation between Ticket and survival.<br>
Cabin feature may be dropped as it is highly incomplete or contains many null values both in training and test dataset.<br>
PassengerId may be dropped from training dataset as it does not contribute to survival.<br>
Name feature is relatively non-standard, may not contribute directly to survival, so maybe dropped.<br>
###### Creating.
We may want to create a new feature called Family based on Parch and SibSp to get total count of family members on board.<br>
We may want to engineer the Name feature to extract Title as a new feature.<br>
We may want to create new feature for Age bands. This turns a continous numerical feature into an ordinal categorical feature.<br>
We may also want to create a Fare range feature if it helps our analysis.<br>
###### Classifying.
We may also add to our assumptions based on the problem description noted earlier.<br>
Women (Sex=female) were more likely to have survived.<br>
Children (Age<?) were more likely to have survived.<br> 
The upper-class passengers (Pclass=1) were more likely to have survived.<br>

<a id='#Missing-data'></a>

# Missing data

[[ go back to the top ]](#Table-of-contents)<br>
Some predictive models inherently are able to deal with missing data (neural networks come to mind) and others require that the missing values be dealt with separately.<br>
### Identifying Missing data
We can use seaborn to create a simple heatmap to see where we are missing data!<br>
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')<br>

### Taking Care with Missing values
1) Throw out any data with missing values<br>
If you’ve got a lot of data that isn’t missing any values it is certainly the quickest and easiest way to handle it.
<br>
train.drop('Cabin',axis=1,inplace=True)<br>
2) Assign a value that indicates a missing value<br>
This is particularly appropriate for categorical variables (more on this in the next post). I really like using this approach when possible because the fact that the value is missing can be useful information in and of itself. Perhaps when a value is missing for a particular variable, that has some underlying cause that makes it correlate more highly with another value.One interesting trick is that you can do this with binary variables by setting the false value as -1, the true value as 1, and missing values as 0.<br>
df['Cabin'][df.Cabin.isnull()] = 'U0'<br>
3) Assign the average value<br>
This is a very common approach because it is simple, and for variables that aren’t extremely important it very well may be good enough. You can also incorporate other variables to create subsets and assign the average within the group. In cases of categorical variables, the most common value can be applied rather than the statistical mean.<br>
df['Fare'][ np.isnan(df['Fare']) ] = df['Fare'].median()<br>
df.Embarked[ df.Embarked.isnull() ] = df.Embarked.dropna().mode().values <br>
4) Use a regression or another simple model to predict the values of missing variables <br>
Useful code :<br>
from sklearn.ensemble import RandomForestRegressor (Can be LinearRegression as well) <br>
 
def setMissingAges(df):
    
    # Grab all the features that can be included in a Random Forest Regressor
    age_df = df[['Age','Embarked','Fare', 'Parch', 'SibSp', 'Title_id','Pclass','Names','CabinLetter']]
    
    # Split into sets with known and unknown Age values
    knownAge = age_df.loc[ (df.Age.notnull()) ]
    unknownAge = age_df.loc[ (df.Age.isnull()) ]
    
    # All age values are stored in a target array
    y = knownAge.values[:, 0]
    
    # All the other values are stored in the feature array
    X = knownAge.values[:, 1::]
    
    # Create and fit a model
    rtr = RandomForestRegressor(n_estimators=2000, n_jobs=-1)
    rtr.fit(X, y)
    
    # Use the fitted model to predict the missing values
    predictedAges = rtr.predict(unknownAge.values[:, 1::])
    
    # Assign those predictions to the full data set
    df.loc[ (df.Age.isnull()), 'Age' ] = predictedAges 
    
    return df
<br>
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):

        if Pclass == 1:
            return 37

        elif Pclass == 2:
            return 29

        else:
            return 24

    else:
        return Age
train['Age'] = train[['Age','Pclass']].apply(impute_age,axis=1)

5)Impute missing data using a reliable method, such as k-nearest neighbors.<br>
<br>
from sklearn.preprocessing import Imputer <br>
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)<br>
X = imp.fit_transform(X)<br>
<br>
The Imputer class provides basic strategies for imputing missing values, either using the mean, the median or the most frequent value of the row or column in which the missing values are located. This class also allows for different missing values encodings.

### Recheck if Missing data is dealt with
We can use seaborn to create the heatmap again to see whether we are still missing data.<br>
sns.heatmap(train.isnull(),yticklabels=False,cbar=False,cmap='viridis')<br>

<a id='#EDA'></a>

# Exploratory Data Analysis

[[ go back to the top ]](#Table-of-contents)<br>
This is best done by Visualizations<br>
### Univariate Analysis
##### What is the plan ? Based on what happened so far?
<br>
#### Visualizations
##### Histograms <br>
A fast way to get an idea of the distribution of each attribute is to look at histograms. Histograms group data into bins and provide you a count of the number of observations in each bin. From the shape of the bins you can quickly get a feeling for whether an attribute is Gaussian’, skewed or even has an exponential distribution. It can also help you see possible outliers.<br>
Bins should be all the same size. For example, groups of ten or a hundred.<br>
Bins should include all of the data, even outliers. If your outliers fall way outside of your other data, consider lumping them in with your first or last bin. This creates a rough histogram — make sure you note where outliers are being included.<br>
Boundaries for bins should land at whole numbers whenever possible (this makes the chart easier to read).<br>
Choose between 5 and 20 bins. The larger the data set, the more likely you’ll want a large number of bins. For example, a set of 12 data pieces might warrant 5 bins but a set of 1000 numbers will probably be more useful with 20 bins. The exact number of bins is usually a judgment call.<br>
If at all possible, try to make your data set evenly divisible by the number of bins. For example, if you have 10 pieces of data, work with 5 bins instead of 6 or 7.<br>
One goal would be to minimize the integrated mean square error.  When the distribution is normal, this can be done with the Freedman–Diaconis rule of course, one might listent to William S. Cleveland (perhaps the world's leading expert on statistical graphics) and not use histograms at all.
Useful code : <br>
sns.set_style('whitegrid')<br>
sns.countplot(x='Survived',data=train,palette='RdBu_r')<br>
train['Age'].hist(bins=30,color='darkred',alpha=0.7)<br>
sns.distplot(train['Age'].dropna(),kde=False,color='darkred',bins=30)<br>
train['Fare'].hist(color='green',bins=40,figsize=(8,4))
##### Density Plots
Density plots are another way of getting a quick idea of the distribution of each attribute. The plots look like an abstracted histogram with a smooth curve drawn through the top of each bin, much like your eye tried to do with the histograms.<br>
sns.distplot(train['Age'].dropna(),kde=True,color='darkred',bins=30)<br>
##### Box and Whisker Plots
Another useful way to review the distribution of each attribute is to use Box and Whisker Plots or boxplots for short.
Boxplots summarize the distribution of each attribute, drawing a line for the median (middle value) and a box around the 25th and 75th percentiles (the middle 50% of the data). The whiskers give an idea of the spread of the data and dots outside of the whiskers show candidate outlier values (values that are 1.5 times greater than the size of spread of the middle 50% of the data).<br>
plt.figure(figsize=(12, 7))<br>
sns.boxplot(x='Pclass',y='Age',data=train,palette='winter')

<a id='#Takeaway2'></a>

### Takeaway from "Univariate Analysis"
What did I learn? <br>
[[ go back to the top ]](#Table-of-contents)
##### What is the distribution of numerical feature values across the samples?
This helps us determine, among other early insights, how representative is the training dataset of the actual problem domain.<br>
Total samples are 891 or 40% of the actual number of passengers on board the Titanic (2,224).<br>
Survived is a categorical feature with 0 or 1 values.<br>
Around 38% samples survived representative of the actual survival rate at 32%.<br>
Most passengers (> 75%) did not travel with parents or children.<br>
Nearly 30% of the passengers had siblings and/or spouse aboard.<br>
Fares varied significantly with few passengers ( < 1 percent) paying as high as 512.<br>
Few elderly passengers ( < 1 percent) within age range 65-80. <br>
##### What is the distribution of categorical feature values across the samples?
Names are unique across the dataset (count=unique=891)<br>
Sex variable as two possible values with 65% male (top=male, freq=577/count=891).<br>
Cabin values have several dupicates across samples. Alternatively several passengers shared a cabin.<br>
Embarked takes three possible values. S port used by most passengers (top=S)<br>
Ticket feature has high ratio (22%) of duplicate values (unique=681).<br>
##### Can anomalies/outliers/anything weird be spotted?

##### If yes(for the previous question) ,what is the plan to handle anomalies/outliers/anything weird that has be spotted?

### Assumtions based on data analysis
We arrive at following assumptions based on data analysis done so far. We may validate these assumptions further before taking appropriate actions.
###### Correlating.
We want to know how well does each feature correlate with Survival. We want to do this early in our project and match these quick correlations with modelled correlations later in the project.
###### Completing.
We may want to complete Age feature as it is definitely correlated to survival.<br>
We may want to complete the Embarked feature as it may also correlate with survival or another important feature.
###### Correcting.
Ticket feature may be dropped from our analysis as it contains high ratio of duplicates (22%) and there may not be a correlation between Ticket and survival.<br>
Cabin feature may be dropped as it is highly incomplete or contains many null values both in training and test dataset.<br>
PassengerId may be dropped from training dataset as it does not contribute to survival.<br>
Name feature is relatively non-standard, may not contribute directly to survival, so maybe dropped.<br>
###### Creating.
We may want to create a new feature called Family based on Parch and SibSp to get total count of family members on board.<br>
We may want to engineer the Name feature to extract Title as a new feature.<br>
We may want to create new feature for Age bands. This turns a continous numerical feature into an ordinal categorical feature.<br>
We may also want to create a Fare range feature if it helps our analysis.<br>
###### Classifying.
We may also add to our assumptions based on the problem description noted earlier.<br>
Women (Sex=female) were more likely to have survived.<br>
Children (Age<?) were more likely to have survived.<br> 
The upper-class passengers (Pclass=1) were more likely to have survived.<br>

<a id='#Multivariate'></a>

### Multivariate Analysis
[[ go back to the top ]](#Table-of-contents)<br>
##### What is the plan ? Based on what happened so far?
<br>
This is best done by Visualizations and by feature correlations by pivoting features against each other<br>
##### Analyze by pivoting features
To confirm some of our observations and assumptions, we can quickly analyze our feature correlations by pivoting features against each other.<br>
It also makes sense doing so only for features which are categorical (Sex), ordinal (Pclass) or discrete (SibSp, Parch) type.<br>
Pclass We observe significant correlation (>0.5) among Pclass=1 and Survived (classifying # 3). We decide to include this feature in our model.<br>
Sex We confirm the observation during problem definition that Sex=female had very high survival rate at 74% (classifying # 1).<br>
SibSp and Parch These features have zero correlation for certain values. It may be best to derive a feature or a set of features from these individual features (creating # 1).<br>
Sample code :<br>
train_df[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)
##### Visualizations
For<b> Large Datasets use Dimentionality Reduction Techniques </b> like PCA and t-SNE to viaualize data
<br>
###### PCA
from sklearn.preprocessing import StandardScaler<br>
scaler = StandardScaler()<br>
scaler.fit(df)<br>
scaled_data = scaler.transform(df)<br>
from sklearn.decomposition import PCA<br>
pca = PCA(n_components=2)<br>
pca.fit(scaled_data)<br>
x_pca = pca.transform(scaled_data)<br>
scaled_data.shape<br>
x_pca.shape<br>
plt.figure(figsize=(8,6))<br>
plt.scatter(x_pca[:,0],x_pca[:,1],c=cancer['target'],cmap='plasma')<br>
plt.xlabel('First principal component')<br>
plt.ylabel('Second Principal Component')<br>
#The components correspond to combinations of the original features, the components themselves are stored as an attribute of the fitted PCA object:<br>
pca.components_ <br>
df_comp = pd.DataFrame(pca.components_,columns=cancer['feature_names'])<br>
plt.figure(figsize=(12,6))<br>
sns.heatmap(df_comp,cmap='plasma',)<br>
This heatmap and the color bar basically represent the correlation between the various feature and the principal component itself.<br>
###### t-SNE 
When PCA doesnt work ..when we need a non-linear dimentionality reduction.It is highly recommended to use another dimensionality reduction method (e.g. PCA for dense data or TruncatedSVD for sparse data) to reduce the number of dimensions to a reasonable amount (e.g. 50) if the number of features is very high. This will suppress some noise and speed up the computation of pairwise distances between samples.<br>
from sklearn.manifold import TSNE<br>
X_tsne = TSNE(learning_rate=100).fit_transform(iris.data)<br>
X_pca = PCA().fit_transform(iris.data)<br>
figure(figsize=(10, 5))<br>
subplot(121)<br>
scatter(X_tsne[:, 0], X_tsne[:, 1], c=iris.target)<br>
subplot(122)<br>
scatter(X_pca[:, 0], X_pca[:, 1], c=iris.target)<br>
(Vary the colour of the points with different features...Color-coherent clusters shows that for T-SNE, nothing is closer to a given observation than an observation with that the similar feature . This is also a consequence of the T-SNE algorithm itself, that focuses on preserving very small pairwise distances between points, contrary to PCA.To spot abnormal points : those with color that do not correspond to their cluster. They are anomalies.<br>
###### Pairplots (for small datasets)
sns.pairplot(df,hue='Kyphosis',palette='Set1')<br>
sns.pairplot(iris,hue='species',palette='Dark2')<br>
sb.pairplot(iris_data.dropna(), hue='class')
<br>
scatter_matrix(dataset)<br>
plt.show()<br>
###### Countplots with hue
sns.set_style('whitegrid')<br>
sns.countplot(x='Survived',hue='Sex',data=train,palette='RdBu_r')<br>
ns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data=train,palette='rainbow')<br>
###### Jointplot (scatterplot)
sns.jointplot(x='fico',y='int.rate',data=loans,color='purple')<br>
sns.jointplot(x='Time on Website',y='Yearly Amount Spent',data=customers)<br>
sns.jointplot(x='Time on App',y='Length of Membership',kind='hex',data=customers)<br>
<br>
plt.scatter(data[0][:,0],data[0][:,1],c=data[1],cmap='rainbow')<br>
<br>
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True,figsize=(10,6))<br>
ax1.set_title('K Means')<br>
ax1.scatter(data[0][:,0],data[0][:,1],c=kmeans.labels_,cmap='rainbow')<br>
ax2.set_title("Original")<br>
ax2.scatter(data[0][:,0],data[0][:,1],c=data[1],cmap='rainbow')<br>
###### lmplots
plt.figure(figsize=(11,7))<br>
sns.lmplot(y='int.rate',x='fico',data=loans,hue='credit.policy',col='not.fully.paid',palette='Set1')<br>
<br>
sns.set_style('whitegrid')<br>
sns.lmplot('Room.Board','Grad.Rate',data=df, hue='Private',<br>
           palette='coolwarm',size=6,aspect=1,fit_reg=False)<br>
<br>
###### Overlapping histograms
ns.set_style('darkgrid')<br>
g = sns.FacetGrid(df,hue="Private",palette='coolwarm',size=6,aspect=2)<br>
g = g.map(plt.hist,'Outstate',bins=20,alpha=0.7)<br>
<br>
###### Heatmaps
sns.heatmap(USAhousing.corr())<br>
<br>
plt.figure(figsize=(12,6))<br>
sns.heatmap(df_comp,cmap='plasma',)
<br>
###### kdeplot
sns.kdeplot( setosa['sepal_width'], setosa['sepal_length'],<br>
                 cmap="plasma", shade=True, shade_lowest=False)
<br>
###### violinplot
plt.figure(figsize=(10, 10))<br>

for column_index, column in enumerate(iris_data_clean.columns):<br>
    if column == 'class': <br>
        continue <br>
    plt.subplot(2, 2, column_index + 1) <br>
    sb.violinplot(x='class', y=column, data=iris_data_clean) <br>

<a id='#Takeaway3'></a>

### Takeaway from "Multivariate Analysis"
What did I learn? <br>
[[ go back to the top ]](#Table-of-contents)
##### What is the distribution of numerical feature values across the samples?
This helps us determine, among other early insights, how representative is the training dataset of the actual problem domain.<br>
Total samples are 891 or 40% of the actual number of passengers on board the Titanic (2,224).<br>
Survived is a categorical feature with 0 or 1 values.<br>
Around 38% samples survived representative of the actual survival rate at 32%.<br>
Most passengers (> 75%) did not travel with parents or children.<br>
Nearly 30% of the passengers had siblings and/or spouse aboard.<br>
Fares varied significantly with few passengers ( < 1 percent) paying as high as 512.<br>
Few elderly passengers ( < 1 percent) within age range 65-80. <br>
##### What is the distribution of categorical feature values across the samples?
Names are unique across the dataset (count=unique=891)<br>
Sex variable as two possible values with 65% male (top=male, freq=577/count=891).<br>
Cabin values have several dupicates across samples. Alternatively several passengers shared a cabin.<br>
Embarked takes three possible values. S port used by most passengers (top=S)<br>
Ticket feature has high ratio (22%) of duplicate values (unique=681).<br>
##### Can anomalies/outliers/anything weird be spotted?

##### If yes(for the previous question) ,what is the plan to handle anomalies/outliers/anything weird that has be spotted?
### Assumtions based on data analysis
We arrive at following assumptions based on data analysis done so far. We may validate these assumptions further before taking appropriate actions.
###### Correlating.
We want to know how well does each feature correlate with Survival. We want to do this early in our project and match these quick correlations with modelled correlations later in the project.
###### Completing.
We may want to complete Age feature as it is definitely correlated to survival.<br>
We may want to complete the Embarked feature as it may also correlate with survival or another important feature.
###### Correcting.
Ticket feature may be dropped from our analysis as it contains high ratio of duplicates (22%) and there may not be a correlation between Ticket and survival.<br>
Cabin feature may be dropped as it is highly incomplete or contains many null values both in training and test dataset.<br>
PassengerId may be dropped from training dataset as it does not contribute to survival.<br>
Name feature is relatively non-standard, may not contribute directly to survival, so maybe dropped.<br>
###### Creating.
We may want to create a new feature called Family based on Parch and SibSp to get total count of family members on board.<br>
We may want to engineer the Name feature to extract Title as a new feature.<br>
We may want to create new feature for Age bands. This turns a continous numerical feature into an ordinal categorical feature.<br>
We may also want to create a Fare range feature if it helps our analysis.<br>
###### Classifying.
We may also add to our assumptions based on the problem description noted earlier.<br>
Women (Sex=female) were more likely to have survived.<br>
Children (Age<?) were more likely to have survived.<br> 
The upper-class passengers (Pclass=1) were more likely to have survived.<br>


<a id='#Outliers'></a>

### Outliers
[[ go back to the top ]](#Table-of-contents)<br>
#### Here are some changes you can make to your model:

<b>Use a model that's resistant to outliers.</b><br>
Tree-based models are generally not as affected by outliers, while regression-based models are. If you're performing a statistical test, try a non-parametric test instead of a parametric one.
<br>
<b>Use a more robust error metric.</b><br>
Switching from mean squared error to mean absolute difference (or something like Huber Loss) reduces the influence of outliers. Why is the median a measure of central tendency?
<br>
<b>Here are some changes you can make to your data:</b><br>
<br>
Winsorize your data. Artificially cap your data at some threshold. <br>
Transform your data. If your data has a very pronounced right tail, try a log transformation.<br>
Remove the outliers. This works if there are very few of them and you're fairly certain they're anomalies and not worth predicting.<br>

<a id='#Feature'></a>

# Feature Engineering

[[ go back to the top ]](#Table-of-contents)

### Feature Engineering : Add new features

[[ go back to the top ]](#Table-of-contents)
##### Derived Features
Very basic examples of a useful derived variable might be pulling the country code and/or area code out of telephone numbers, or extracting country/state/city from GPS coordinates. Any time a qualitative variable represents an object in the world that we know something about, there is an opportunity to derive variables from it. Also, if a data set represents a timeseries or other historical behavioral information that can also provide a great opportunity for uncovering derived variables.<br>
##### Decomposition
There may be features that represent a complex concept that may be more useful to a machine learning method when split into the constituent parts. An example is a date that may have day and time components that in turn could be split out further. Perhaps only the hour of day is relevant to the problem being solved. consider what feature decompositions you can perform.
##### Aggregation
There may be features that can be aggregated into a single feature that would be more meaningful to the problem you are trying to solve. For example, there may be a data instances for each time a customer logged into a system that could be aggregated into a count for the number of logins allowing the additional instances to be discarded. Consider what type of feature aggregations could perform.
<br> Examples - 
###### 1. Create variables for difference in date, time and addresses<br>
While you might be using date and time values on their own, you can create new variables by considering differences in dates and time. Here is an example hypothesis: An applicant who takes days to fill in an application form is likely to be less interested / motivated in the product compared to some one who fills in the same application with in 30 minutes. Similarly, for a bank, time elapsed between dispatch of login details for Online portal and customer logging in might show customers’ willingness to use Online portal.<br>
<br>
Another example is that a customer living closer to a bank branch is more likely to have a higher engagement than a customer living far off.<br>
<br>
###### 2. Create new ratios and proportions<br>
Instead of just keeping past inputs and outputs in your dataset, creating ratios out of them might add a lot of value. Some of the ratios, I have used in past are: Input / Output (past performance), productivity, efficiency and percentages. For example, in order to predict future performance of credit card sales of a branch, ratios like credit card sales / Sales person or Credit card Sales / Marketing spend would be more powerful than just using absolute number of card sold in the branch<br>
###### 3. Include effect of influencer<br>
Influencer can impact on the behaviour of your study significantly. Influencer could be of various form and sizes. It could be an employee of your Organization, agent of your Organization or a customer of your Organization. Bringing the impact of these related entities can improve the models significantly. For example, a loan initiated by a sub-set of brokers (and not all brokers) might be more likely to be transferred to a different entity after the lock-in period. Similarly, there might be a sub set of Sales personnel involved who do a higher cross-sell to their customers.<br>
<br>
###### 4. Check variables for seasonality and create the model for right period<br>
A lot of businesses face some kind of seasonality. It could be driven by tax benefits, festive season or weather. If this is the case, you need to make sure that the data and variables are chosen for the right period.<br>

<a id='#Feature1'></a>

### Feature Engineering : Remove Features

[[ go back to the top ]](#Table-of-contents)
<b>Useless Attributes</b>
<b>Correlated Attributes:</b> Some algorithms degrade in importance with the existence of highly correlated attributes. Pairwise attributes with high correlation can be identified and the most correlated attributes can be removed from the data.<br>

<a id='#Feature12'></a>

### Feature Engineering : Apply Transformations

[[ go back to the top ]](#Table-of-contents)
When we can transform complex non-linear relationships into linear relationships. Existence of a linear relationship between variables is easier to comprehend compared to a non-linear or curved relation. Transformation helps us to convert a non-linear relation into linear relation. Scatter plot can be used to find the relationship between two continuous variables. These transformations also improve the prediction. Log transformation is one of the commonly used transformation technique used in these situations.<br>
Symmetric distribution is preferred over skewed distribution as it is easier to interpret and generate inferences. Some modeling techniques requires normal distribution of variables. So, whenever we have a skewed distribution, we can use transformations which reduce skewness. For right skewed distribution, we take square / cube root or logarithm of variable and for left skewed, we take square / cube or exponential of variables.<br>
Variable Transformation is also done from an implementation point of view (Human involvement). Let’s understand it more clearly. In one of my project on employee performance, I found that age has direct correlation with performance of the employee i.e. higher the age, better the performance. From an implementation stand point, launching age based progamme might present implementation challenge. However, categorizing the sales agents in three age group buckets of < 30 years, 30-45 years and >45  and then formulating three different strategies for each group is a judicious approach. This categorization technique is known as Binning of Variables.<br>
There are various methods used to transform variables. As discussed, some of them include square root, cube root, logarithmic, binning, reciprocal and many others. Let’s look at these methods in detail by highlighting the pros and cons of these transformation methods.<br>
<b>Logarithm: </b>Log of a variable is a common transformation method used to change the shape of distribution of the variable on a distribution plot. It is generally used for reducing right skewness of variables. Though, It can’t be applied to zero or negative values as well.Log(Marketing spend) might have a more representable relationship with Sales as compared to absolute Marketing spend.<br>
<b>Square / Cube root: </b>The square and cube root of a variable has a sound effect on variable distribution. However, it is not as significant as logarithmic transformation. Cube root has its own advantage. It can be applied to negative <b>Binning: </b>It is used to categorize variables. It is performed on original values, percentile or frequency. Decision of categorization technique is based on business understanding. For example, we can categorize income in three categories, namely: High, Average and Low. We can also perform co-variate binning which depends on the value of more than one variables.<br>

<a id='#Feature2'></a>

### Feature Engineering : Convert Categorical Variables 

[[ go back to the top ]](#Table-of-contents)

Logistic regression, distance based methods such as kNN, support vector machines, tree based methods etc. in sklearn needs numeric arrays. Features having string values cannot be handled by these learners.<br>
Sklearn provides a very efficient tools for encoding the levels of a categorical features into numeric values.<br>
##### LabelEncoder
LabelEncoder encode labels with value between 0 and n_classes-1.<br>
Lets encode all the categorical features.<br>

from sklearn.preprocessing import LabelEncoder<br>
le=LabelEncoder()<br>
for col in X_test.columns.values:<br>
       # Encoding only categorical variables
       if X_test[col].dtypes=='object':
       #Using whole data to form an exhaustive list of levels
       data=X_train[col].append(X_test[col])
       le.fit(data.values)
       X_train[col]=le.transform(X_train[col])
       X_test[col]=le.transform(X_test[col])
<br>
###### One-hot-encoding
       from sklearn.preprocessing import OneHotEncoder<br>
       enc=OneHotEncoder(sparse=False)
       X_train_1=X_train
       X_test_1=X_test
       columns=['Gender', 'Married', 'Dependents', 'Education','Self_Employed',
          'Credit_History', 'Property_Area']
       for col in columns:
       # creating an exhaustive list of all possible categorical values
       data=X_train[[col]].append(X_test[[col]])
       enc.fit(data)
       # Fitting One Hot Encoding on train data
       temp = enc.transform(X_train[[col]])
       # Changing the encoded features into a data frame with new column names
       temp=pd.DataFrame(temp,columns=[(col+"_"+str(i)) for i in data[col]
            .value_counts().index])
       # In side by side concatenation index values should be same
       # Setting the index values similar to the X_train data frame
       temp=temp.set_index(X_train.index.values)
       # adding the new One Hot Encoded varibales to the train data frame
       X_train_1=pd.concat([X_train_1,temp],axis=1)
       # fitting One Hot Encoding on test data
       temp = enc.transform(X_test[[col]])
       # changing it into data frame and adding column names
       temp=pd.DataFrame(temp,columns=[(col+"_"+str(i)) for i in data[col]
            .value_counts().index])
       # Setting the index for proper concatenation
       temp=temp.set_index(X_test.index.values)
       # adding the new One Hot Encoded varibales to test data frame
       X_test_1=pd.concat([X_test_1,temp],axis=1)
       
There are some cases where LabelEncoder or DictVectorizor are useful, but these are quite limited in my opinion due to ordinality.
LabelEncoder can turn [dog,cat,dog,mouse,cat] into [1,2,1,3,2], but then the imposed ordinality means that the average of dog and mouse is cat. Still there are algorithms like decision trees and random forests that can work with categorical variables just fine and LabelEncoder can be used to store values using less disk space.
One-Hot-Encoding has a the advantage that the result is binary rather than ordinal and that everything sits in an orthogonal vector space. The disadvantage is that for high cardinality, the feature space can really blow up quickly and you start fighting with the curse of dimensionality. In these cases, I typically employ one-hot-encoding followed by PCA for dimensionality reduction. I find that the judicious combination of one-hot plus PCA can seldom be beat by other encoding schemes. PCA finds the linear overlap, so will naturally tend to group similar features into the same feature.

<a id='#Feature3'></a>

### Feature Engineering : Standardize

[[ go back to the top ]](#Table-of-contents)
Standardization is a useful technique to transform attributes with a Gaussian distribution and differing means and standard deviations to a standard Gaussian distribution with a mean of 0 and a standard deviation of 1.
It is most suitable for techniques that assume a Gaussian distribution in the input variables and work better with rescaled data, such as linear regression, logistic regression and linear discriminate analysis.Elements such as l1 ,l2 regularizer in linear models (logistic comes under this category) and RBF kernel in SVM in objective function of learners assumes that all the features are centered around zero and have variance in the same order.<br>
<br>
#Standardize data (0 mean, 1 stdev)<br>
from sklearn.preprocessing import StandardScaler<br>
scaler = StandardScaler().fit(X)<br>
rescaledX = scaler.transform(X)<br>
#summarize transformed data<br>
numpy.set_printoptions(precision=3)<br>
print(rescaledX[0:5,:])<br>
Standardizing the data when using a estimator having l1 or l2 regularization helps us to increase the accuracy of the prediction model. Other learners like kNN with euclidean distance measure, k-means, SVM, perceptron, neural networks, linear discriminant analysis, principal component analysis may perform better with standardized data.<br>

<a id='#Feature4'></a>

### Feature Engineering : Normalize

[[ go back to the top ]](#Table-of-contents)
Normalizing in scikit-learn refers to rescaling each observation (row) to have a length of 1 (called a unit norm in linear algebra).
This preprocessing can be useful for sparse datasets (lots of zeros) with attributes of varying scales when using algorithms that weight input values such as neural networks and algorithms that use distance measures such as K-Nearest Neighbors.<br>
#Normalize data (length of 1<br>
from sklearn.preprocessing import Normalizer<br>
scaler = Normalizer().fit(X)<br>
normalizedX = scaler.transform(X)<br>
#summarize transformed data<br>
numpy.set_printoptions(precision=3)<br>
print(normalizedX[0:5,:])<br>

<a id='#Feature5'></a>

### Feature Engineering : Make Binary

[[ go back to the top ]](#Table-of-contents)
You can transform your data using a binary threshold. All values above the threshold are marked 1 and all equal to or below are marked as 0. This is called binarizing your data or threshold your data. It can be useful when you have probabilities that you want to make crisp values. It is also useful when feature engineering and you want to add new features that indicate something meaningful. <br>
<br>
from sklearn.preprocessing import Binarizer<br>
binarizer = Binarizer(threshold=0.0).fit(X)<br>
binaryX = binarizer.transform(X)<br>
#summarize transformed data<br>
numpy.set_printoptions(precision=3)<br>
print(binaryX[0:5,:])<br>

<a id='#Feature6'></a>

### Feature Engineering : Scale

[[ go back to the top ]](#Table-of-contents)
When your data is comprised of attributes with varying scales, many machine learning algorithms can benefit from rescaling the attributes to all have the same scale.
Often this is referred to as normalization and attributes are often rescaled into the range between 0 and 1. This is useful for optimization algorithms in used in the core of machine learning algorithms like gradient descent. It is also useful for algorithms that weight inputs like regression and neural networks and algorithms that use distance measures like K-Nearest Neighbors.<br>
<br>
#Rescale data (between 0 and 1)<br>
from sklearn.preprocessing import MinMaxScaler<br>
array = data.values<br>
#separate array into input and output components<br>
X = array[:,0:8]<br>
Y = array[:,8]<br>
scaler = MinMaxScaler(feature_range=(0, 1))<br>
rescaledX = scaler.fit_transform(X)<br>
#summarize transformed data<br>
numpy.set_printoptions(precision=3)<br>
print(rescaledX[0:5,:])<br>

<a id='#Feature7'></a>

### Feature Engineering : Bin

[[ go back to the top ]](#Table-of-contents)
Binning is a term used to indicate creating quantiles. This allows you to create an ordered, categorical variable out of a range of values. In algorithms that respond effectively use categorical information this can be useful (probably not so great for linear regression).<br>
<br>
#Divide all fares into quartiles<br>
df['Fare_bin'] = pd.qcut(df['Fare'], 4)<br> 
#qcut() creates a new variable that identifies the quartile range, but we can't use the string so either<br>
#factorize or create dummies from the result<br>
df['Fare_bin_id'] = pd.factorize(df['Fare_bin']) <br>
df = pd.concat([df, pd.get_dummies(df['Fare_bin']).rename(columns=lambda x: 'Fare_' + str(x))], axis=1)<br>

<a id='#Feature8'></a>

### Feature Engineering : Correlation and Interaction variables

[[ go back to the top ]](#Table-of-contents)
Interaction variables capture effects of the relationship between variables. They are constructed by performing mathematical operations on sets of features. The simple approach that we use in this example is to perform basic operators (add, subtract, multiply, divide) on each pair of numerical features. We could also get much more involved and include more than 2 features in each calculation, and/or use other operators (sqrt, ln, trig functions, etc).<br>
This process of automated feature generation can quickly produce a LOT of new variables. In our case, we use 9 features to generate 176 new interaction features. In a larger data set with dozens or hundreds of numeric features, this process can generate an overwhelming number of new interactions. Some types of models are really good at handling a very large number of features (I’ve heard of thousands to millions), which would be necessary in such a case.
It’s very likely that some of the new interaction variables are going to be highly correlated with one of their original variables, or with other interactions, which can be a problem especially for linear models. Highly correlated variables can cause an issue called “multicollinearity”. There is a lot of information out there about how to identify, deal with, and safely ignore multicollinearity in a data set so I’ll avoid an explanation here, but I’ve included some great links at the bottom of this post if you’re interested.<br> 

<a id='#Feature9'></a>

### Feature Engineering : Testing

[[ go back to the top ]](#Table-of-contents)
we can similarly set up unit tests to verify our expectations about a data set.
We can quickly test our data using assert statements: We assert that something must be true, and if it is, then nothing happens and the notebook continues running. However, if our assertion is wrong, then the notebook stops running and brings it to our attention. For example:.<br>
#We know that we should only have three classes
assert len(iris_data_clean['class'].unique()) == 3
#We know that sepal lengths for 'Iris-versicolor' should never be below 2.5 cm
assert iris_data_clean.loc[iris_data_clean['class'] == 'Iris-versicolor', 'sepal_length_cm'].min() >= 2.5
#We know that our data set should have no missing measurements
assert len(iris_data_clean.loc[(iris_data_clean['sepal_length_cm'].isnull()) |
                               (iris_data_clean['sepal_width_cm'].isnull()) |
                               (iris_data_clean['petal_length_cm'].isnull()) |
                               (iris_data_clean['petal_width_cm'].isnull())]) == 0

<a id='#Feature10'></a>

### Feature Engineering : Repeat EDA 

[[ go back to the top ]](#Table-of-contents)


<a id='#Feature11'></a>

### Feature Engineering : Takeaway

[[ go back to the top ]](#Table-of-contents)


<a id='#Feature-Selection'></a>

# Feature Selection

[[ go back to the top ]](#Table-of-contents)

The data features that you use to train your machine learning models have a huge influence on the performance you can achieve.<br>
Irrelevant or partially relevant features can negatively impact model performance.<br>
There are automatic feature selection techniques that you can use to prepare your machine learning data in python with scikit-learn.<br>
Feature selection is a process where you automatically select those features in your data that contribute most to the prediction variable or output in which you are interested. Having irrelevant features in your data can decrease the accuracy of many models, especially linear algorithms like linear and logistic regression.<br>
Three benefits of performing feature selection before modeling your data are:<br>
Reduces Overfitting: Less redundant data means less opportunity to make decisions based on noise.<br>
Improves Accuracy: Less misleading data means modeling accuracy improves.<br>
Reduces Training Time: Less data means that algorithms train faster.<br>
<br>
##### 1. Univariate Selection<br>
Statistical tests can be used to select those features that have the strongest relationship with the output variable.
The scikit-learn library provides the SelectKBest class that can be used with a suite of different statistical tests to select a specific number of features.<br>
The example below uses the chi squared (chi^2) statistical test for non-negative features to select 4 of the best features from the Pima Indians onset of diabetes dataset.<br>
<br>
#Feature Extraction with Univariate Statistical Tests (Chi-squared for classification)<br>
from sklearn.feature_selection import SelectKBest<br>
from sklearn.feature_selection import chi2<br>
array = data.values<br>
X = array[:,0:8]<br>
Y = array[:,8]<br>
#feature extraction<br>
test = SelectKBest(score_func=chi2, k=4)<br>
fit = test.fit(X, Y)<br>
#summarize scores<br>
numpy.set_printoptions(precision=3)<br>
print(fit.scores_)<br>
features = fit.transform(X)<br>
#summarize selected features<br>
print(features[0:5,:])<br>
<br>
[  111.52   1411.887    17.605    53.108  2175.565   127.669     5.393<br>
   181.304]<br>
[[ 148.     0.    33.6   50. ]<br>
 [  85.     0.    26.6   31. ]<br>
 [ 183.     0.    23.3   32. ]<br>
 [  89.    94.    28.1   21. ]<br>
 [ 137.   168.    43.1   33. ]]<br>
<br>
You can see the scores for each attribute and the 4 attributes chosen (those with the highest scores): plas, test, mass and age.
<br>
##### 2. Recursive Feature Elimination
The Recursive Feature Elimination (or RFE) works by recursively removing attributes and building a model on those attributes that remain.<br>
It uses the model accuracy to identify which attributes (and combination of attributes) contribute the most to predicting the target attribute.<br>
You can learn more about the RFE class in the scikit-learn documentation.<br>
The example below uses RFE with the logistic regression algorithm to select the top 3 features. The choice of algorithm does not matter too much as long as it is skillful and consistent.<br>
<br>
from sklearn.feature_selection import RFE<br>
from sklearn.linear_model import LogisticRegression<br>
array = data.values<br>
X = array[:,0:8]<br>
Y = array[:,8]<br>
model = LogisticRegression()<br>
rfe = RFE(model, 3)<br>
fit = rfe.fit(X, Y)<br>
print("Num Features: ",fit.n_features_) <br>
print("Selected Features: ",fit.support_)<br>
print("Feature Ranking: ",fit.ranking_)<br>
<br>
Num Features:  3 <br>
Selected Features:  [ True False False False False  True  True False]<br>
Feature Ranking:  [1 2 3 5 6 1 1 4]<br>
<br>
You can see that RFE chose the the top 3 features as preg, mass and pedi.<br>
These are marked True in the support array and marked with a choice “1” in the ranking array.<br>
<br>
##### 3. Principal Component Analysis
Principal Component Analysis (or PCA) uses linear algebra to transform the dataset into a compressed form.<br>
Generally this is called a data reduction technique. A property of PCA is that you can choose the number of dimensions or principal component in the transformed result.<br>
In the example below, we use PCA and select 3 principal components.
<br>
from sklearn.decomposition import PCA<br>
X = array[:,0:8]<br>
Y = array[:,8]<br>
#feature extraction<br>
pca = PCA(n_components=3)<br>
fit = pca.fit(X)<br>
#summarize components<br>
print("Explained Variance: ",fit.explained_variance_ratio_)<br>
print(fit.components_)<br>
<br>
Explained Variance:  [ 0.889  0.062  0.026]<br>
[[ -2.022e-03   9.781e-02   1.609e-02   6.076e-02   9.931e-01   1.401e-02<br>
    5.372e-04  -3.565e-03] <br>
 [  2.265e-02   9.722e-01   1.419e-01  -5.786e-02  -9.463e-02   4.697e-02<br>
    8.168e-04   1.402e-01]<br>
 [ -2.246e-02   1.434e-01  -9.225e-01  -3.070e-01   2.098e-02  -1.324e-01<br>
   -6.400e-04  -1.255e-01]]<br>
<br>
You can see that the transformed dataset (3 principal components) bare little resemblance to the source data.<br>
<br>
#### 4. Feature Importance
Bagged decision trees like Random Forest and Extra Trees can be used to estimate the importance of features.<br>
<br>
from sklearn.ensemble import ExtraTreesClassifier<br>
#feature extraction<br>
model = ExtraTreesClassifier()<br>
model.fit(X, Y)<br>
print(model.feature_importances_)<br>
<br>
[ 0.119  0.226  0.093  0.081  0.073  0.143  0.119  0.146]<br>
<br>
You can see that we are given an importance score for each attribute where the larger score the more important the attribute. The scores suggest at the importance of plas, age and mass.-+ <br>
USEFUL CODE :
<br>
features_list = input_df.columns.values[1::]<br>
X = input_df.values[:, 1::]<br>
y = input_df.values[:, 0]<br>
 <br>
#Fit a random forest with (mostly) default parameters to determine feature importance<br>
forest = RandomForestClassifier(oob_score=True, n_estimators=10000)<br>
forest.fit(X, y)<br>
feature_importance = forest.feature_importances_ <br>
 <br>
#make importances relative to max importance<br>
feature_importance = 100.0 * (feature_importance / feature_importance.max())<br>
 <br>
#A threshold below which to drop features from the final data set. Specifically, this number represents<br>
#the percentage of the most important feature's importance value<br>
fi_threshold = 15<br>
 <br>
#Get the indexes of all features over the importance threshold<br>
important_idx = np.where(feature_importance > fi_threshold)[0]<br>
 <br>
#Create a list of all the feature names above the importance threshold<br>
important_features = features_list[important_idx]<br>
print "n", important_features.shape[0], "Important features(>", fi_threshold, "% of max importance):n",<br> 
        important_features<br>
<br>
#Get the sorted indexes of important features<br>
sorted_idx = np.argsort(feature_importance[important_idx])[::-1]<br>
print "nFeatures sorted by importance (DESC):n", important_features[sorted_idx]<br>
 <br>
#Adapted from http://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html <br>
pos = np.arange(sorted_idx.shape[0]) + .5 <br>
plt.subplot(1, 2, 2) <br>
plt.barh(pos, feature_importance[important_idx][sorted_idx[::-1]], align='center') <br>
plt.yticks(pos, important_features[sorted_idx[::-1]]) <br>
plt.xlabel('Relative Importance') <br>
plt.title('Variable Importance') <br>
plt.draw()<br>
plt.show()<br>
 <br>
#Remove non-important features from the feature set, and reorder those remaining<br>
X = X[:, important_idx][:, sorted_idx]<br>

<a id='#Resampling'></a>

# Algorithm Evaluation With Resampling Methods

[[ go back to the top ]](#Table-of-contents)
### Split into Train and Test Sets
The simplest method that we can use to evaluate the performance of a machine learning algorithm is to use different training and testing datasets.<br>
We can take our original dataset, split it into two parts. Train the algorithm on the first part, make predictions on the second part and evaluate the predictions against the expected results.<br>
The size of the split can depend on the size and specifics of your dataset, although it is common to use 67% of the data for training and the remaining 33% for testing.<br>
This algorithm evaluation technique is very fast. It is ideal for large datasets (millions of records) where there is strong evidence that both splits of the data are representative of the underlying problem. Because of the speed, it is useful to use this approach when the algorithm you are investigating is slow to train.<br>
A downside of this technique is that it can have a high variance. This means that differences in the training and test dataset can result in meaningful differences in the estimate of accuracy.<br>
In the example below we split the data Pima Indians dataset into 67%/33% split for training and test and evaluate the accuracy of a Logistic Regression model.<br>
<br>
#Evaluate using a train and a test set<br>
import sklearn <br>
from sklearn import cross_validation <br>
from sklearn.linear_model import LogisticRegression <br>
test_size = 0.33<br>
seed = 7<br>
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, test_size=test_size, random_state=seed)<br>
model = LogisticRegression()<br>
model.fit(X_train, Y_train)<br>
result = model.score(X_test, Y_test)<br>
print("Accuracy: {}".format(result*100.0))<br>
We can see that the estimated accuracy for the model was approximately 75%. Note that in addition to specifying the size of the split, we also specify the random seed. Because the split of the data is random, we want to ensure that the results are reproducible. By specifying the random seed we ensure that we get the same random numbers each time we run the code.<br>
This is important if we want to compare this result to the estimated accuracy of another machine learning algorithm or the same algorithm with a different configuration. To ensure the comparison was apples-for-apples, we must ensure that they are trained and tested on the same data.<br>  
### K-fold Cross Validation
Cross validation is an approach that you can use to estimate the performance of a machine learning algorithm with less variance than a single train-test set split.<br>
It works by splitting the dataset into k-parts (e.g. k=5 or k=10). Each split of the data is called a fold. The algorithm is trained on k-1 folds with one held back and tested on the held back fold. This is repeated so that each fold of the dataset is given a chance to be the held back test set.<br>
After running cross validation you end up with k different performance scores that you can summarize using a mean and a standard deviation.<br>
The result is a more reliable estimate of the performance of the algorithm on new data given your test data. It is more accurate because the algorithm is trained and evaluated multiple times on different data.<br>
The choice of k must allow the size of each test partition to be large enough to be a reasonable sample of the problem, whilst allowing enough repetitions of the train-test evaluation of the algorithm to provide a fair estimate of the algorithms performance on unseen data. For modest sized datasets in the thousands or tens of thousands of records, k values of 3, 5 and 10 are common.<br>
10-fold cross validation.<br>
num_instances = len(X)<br>
seed = 7v
kfold = cross_validation.KFold(n=num_instances, n_folds=10, random_state=seed)<br>
model = LogisticRegression()<br>
results = cross_validation.cross_val_score(model, X, Y, cv=kfold)<br>
print("Accuracy: ",results.mean()*100.0, results.std()*100.0)<br>
<br>
You can see that we report both the mean and the standard deviation of the performance measure. When summarizing performance measures, it is a good practice to summarize the distribution of the measures, in this case assuming a Gaussian distribution of performance (a very reasonable assumption) and recording the mean and standard deviation.<br>
### Leave One Out Cross Validation
You can configure cross validation so that the size of the fold is 1 (k is set to the number of observations in your dataset). This variation of cross validation is called leave-one-out cross validation.
The result is a large number of performance measures that can be summarized in an effort to give a more reasonable estimate of the accuracy of your model on unseen data. A downside is that it can be a computationally more expensive procedure than k-fold cross validation.<br>
num_folds = 10<br>
num_instances = len(X)<br>
loocv = cross_validation.LeaveOneOut(num_instances)<br>
model = LogisticRegression()<br>
results = cross_validation.cross_val_score(model, X, Y, cv=loocv)<br>
print("Accuracy: ",results.mean()*100.0, results.std()*100.0)<br>
You can see in the standard deviation that the score has more variance than the k-fold cross validation results described above.<br>
### Repeated Random Test-Train Splits
Another variation on k-fold cross validation is to create a random split of the data like the train/test split described above, but repeat the process of splitting and evaluation of the algorithm multiple times, like cross validation.<br>
This has the speed of using a train/test split and the reduction in variance in the estimated performance of k-fold cross validation. You can also repeat the process many more times as need. A down side is that repetitions may include much of the same data in the train or the test split from run to run, introducing redundancy into the evaluation.<br>
The example below splits the data into a 67%/33% train/test split and repeats the process 10 times.<br>
kfold = cross_validation.ShuffleSplit(num_instances, test_size=test_size, random_state=seed)<br>
model = LogisticRegression()<br>
results = cross_validation.cross_val_score(model, X, Y, cv=kfold)<br>
print("Accuracy: ",results.mean()*100.0, results.std()*100.0)<br>
We can see that the distribution of the performance measure is on par with k-fold cross validation above.<br>
### Stratified k-fold cross-validation
Stratified *k*-fold keeps the class proportions the same across all of the folds, which is vital for maintaining a representative subset of our data set. (e.g., so we don't have 100% `Iris setosa` entries in one of the folds.<br>
<br>
import numpy as np<br>
from sklearn.cross_validation import StratifiedKFold<br>

def plot_cv(cv, n_samples):<br>
    masks = []<br>
    for train, test in cv: <br>
        mask = np.zeros(n_samples, dtype=bool)<br>
        mask[test] = 1<br>
        masks.append(mask)<br>
    <br>    
    plt.figure(figsize=(15, 15))
    plt.imshow(masks, interpolation='none')
    plt.ylabel('Fold')
    plt.xlabel('Row #')

plot_cv(StratifiedKFold(all_classes, n_folds=10), len(all_classes))<br>
<br>
##### What Techniques to Use When
Generally k-fold cross validation is the gold-standard for evaluating the performance of a machine learning algorithm on unseen data with k set to 3, 5, or 10.<br>
Using a train/test split is good for speed when using a slow algorithm and produces performance estimates with lower bias when using large datasets.<br>
Techniques like leave-one-out cross validation and repeated random splits can be useful intermediates when trying to balance variance in the estimated performance, model training speed and dataset size.<br>
The best advice is to experiment and find a technique for your problem that is fast and produces reasonable estimates of performance that you can use to make decisions. If in doubt, use 10-fold cross validation.<br>


<a id='#metrics'></a>

# Performance metrics

[[ go back to the top ]](#Table-of-contents)
### Classification Metrics
Classification problems are perhaps the most common type of machine learning problem and as such there are a myriad of metrics that can be used to evaluate predictions for these problems.
In this section we will review how to use the following metrics:
###### Classification Accuracy.
###### Logarithmic Loss.
###### Area Under ROC Curve.
###### Confusion Matrix.
###### Classification Report.
#### 1. Classification Accuracy
Classification accuracy is the number of correct predictions made as a ratio of all predictions made.<br>
This is the most common evaluation metric for classification problems, it is also the most misused. It is really only suitable when there are an equal number of observations in each class (which is rarely the case) and that all predictions and prediction errors are equally important, which is often not the case.<br>
scoring = 'accuracy'
kfold = cross_validation.KFold(n=num_instances, n_folds=10, random_state=seed)
model = LogisticRegression()
results = cross_validation.cross_val_score(model, X, Y, cv=kfold,scoring=scoring)
print("Accuracy: ",results.mean()*100.0, results.std()*100.0)

#### 2. Logarithmic Loss
Logarithmic loss (or logloss) is a performance metric for evaluating the predictions of probabilities of membership to a given class.<br>
The scalar probability between 0 and 1 can be seen as a measure of confidence for a prediction by an algorithm. Predictions that are correct or incorrect are rewarded or punished proportionally to the confidence of the prediction.<br>
scoring = 'log_loss'
results = cross_validation.cross_val_score(model, X, Y, cv=kfold,scoring=scoring)
print("Accuracy: ",results.mean(), results.std())

Smaller logloss is better with 0 representing a perfect logloss. As mentioned above, the measure is inverted to be ascending when using the cross_val_score() function.

##### 3. Area Under ROC Curve
Area under ROC Curve (or AUC for short) is a performance metric for binary classification problems.
The AUC represents a model’s ability to discriminate between positive and negative classes. An area of 1.0 represents a model that made all predictions perfectly. An area of 0.5 represents a model as good as random.<br>
ROC can be broken down into sensitivity and specificity. A binary classification problem is really a trade-off between sensitivity and specificity.<br>
Sensitivity is the true positive rate also called the recall. It is the number instances from the positive (first) class that actually predicted correctly.<br>
Specificity is also called the true negative rate. Is the number of instances from the negative class (second) class that were actually predicted correctly.<br>
An ROC curve is the most commonly used way to visualize the performance of a binary classifier, and AUC is (arguably) the best way to summarize its performance in a single number.<br>
scoring = 'roc_auc'
results = cross_validation.cross_val_score(model, X, Y, cv=kfold,scoring=scoring)
print("Accuracy: ",results.mean(), results.std())
If the AUC is relatively close to 1 and greater than 0.5, suggesting some skill in the predictions

#### 4. Confusion Matrix
The confusion matrix is a handy presentation of the accuracy of a model with two or more classes.
The table presents predictions on the x-axis and accuracy outcomes on the y-axis. The cells of the table are the number of predictions made by a machine learning algorithm.<br>
For example, a machine learning algorithm can predict 0 or 1 and each prediction may actually have been a 0 or 1. Predictions for 0 that were actually 0 appear in the cell for prediction=0 and actual=0, whereas predictions for 0 that were actually 1 appear in the cell for prediction = 0 and actual=1. And so on.<br>
from sklearn.metrics import confusion_matrix
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, test_size=test_size, random_state=seed)
model = LogisticRegression()
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
print(matrix)

#### 5. Classification Report
Scikit-learn does provide a convenience report when working on classification problems to give you a quick idea of the accuracy of a model using a number of measures.<br>
The classification_report() function displays the precision, recall, f1-score and support for each class.
from sklearn.metrics import classification_report
0report = classification_report(Y_test, predicted)
print(report)
<b>Mean F Score</b><br>
The F1 score, commonly used in information retrieval, measures accuracy using the statistics precision p and recall r. Precision is the ratio of true positives (tp) to all predicted positives (tp + fp). Recall is the ratio of true positives to all actual positives (tp + fn). The F1 score is given by <br>
F1= 2*(pr)/(p+r) <br>
  where  p= (tp)/(tp + fp) , r= (tp)/(tp + fn)<br>
<br>
The F1 metric weights recall and precision equally, and a good retrieval algorithm will maximize both precision and recall simultaneously. Thus moderately good performance on both will be favored over extremely good performance on one and poor performance on the other.
For MeanF1Score calculate F1 metrics for each label, and find their average, weighted by support (the number of true instances for each label).

<a id='#Spot-check'></a>

# Spot-check Algorithms

[[ go back to the top ]](#Table-of-contents)
Spot-checking is a way of discovering which algorithms perform well on your machine learning problem.<br>
You cannot know which algorithms are best suited to your problem before hand. You must trial a number of methods and focus attention on those that prove themselves the most promising.<br>
You can guess at what algorithms might do well on your dataset, and this can be a good starting point.<br>

Try a mixture of algorithm representations (e.g. instances and trees).<br>
Try a mixture of learning algorithms (e.g. different algorithms for learning the same type of representation).<br>
Try a mixture of modeling types (e.g. linear and nonlinear functions or parametric and nonparametric).<br>
Let’s get specific. <br>
Algorithms Overview<br>
We are going to take a look at 6 classification algorithms to spot check on your dataset.<br>
##### 2 Linear Machine Learning Algorithms:
Logistic Regression <br>
Linear Discriminant Analysis <br>
###### 4 Nonlinear Machine Learning Algorithms:
K-Nearest Neighbors<br>
Naive Bayes<br>
Classification and Regression Trees<br>
Support Vector Machines<br>
<br>
kfold = cross_validation.KFold(n=num_instances, n_folds=10, random_state=seed)<br>
model = LogisticRegression()<br>
results = cross_validation.cross_val_score(model, X, Y, cv=kfold)<br>
print("Accuracy: ",results.mean()*100.0, results.std()*100.0)<br>

<a id='#Spot-check'></a>

# Model Comparison and Selection

[[ go back to the top ]](#Table-of-contents)
<br>
#prepare models<br>
models = []<br>
models.append(('LR', LogisticRegression()))<br>
models.append(('LDA', LinearDiscriminantAnalysis()))<br>
models.append(('KNN', KNeighborsClassifier()))<br>
models.append(('CART', DecisionTreeClassifier()))<br>
models.append(('NB', GaussianNB()))<br>
models.append(('SVM', SVC()))<br>
#evaluate each model in turn<br>
results = []<br>
names = []<br>
scoring = 'accuracy' <br>
for name, model in models:<br>
    kfold = cross_validation.KFold(n=num_instances, n_folds=10, random_state=seed)<br>
    cv_results = cross_validation.cross_val_score(model, X, Y, cv=kfold, scoring=scoring)<br>
    results.append(cv_results)<br>
    names.append(name)<br>
    print("{} : {} , {}".format(name, cv_results.mean(), cv_results.std()))<br>
#boxplot algorithm comparison<br>
fig = plt.figure()<br>
fig.suptitle('Algorithm Comparison')<br>
ax = fig.add_subplot(111)<br>
plt.boxplot(results)<br>
ax.set_xticklabels(names)<br>
plt.show()<br>

"""

========================

Plotting Learning Curves

========================



On the left side the learning curve of a naive Bayes classifier is shown for

the digits dataset. Note that the training score and the cross-validation score

are both not very good at the end. However, the shape of the curve can be found

in more complex datasets very often: the training score is very high at the

beginning and decreases and the cross-validation score is very low at the

beginning and increases. On the right side we see the learning curve of an SVM

with RBF kernel. We can see clearly that the training score is still around

the maximum and the validation score could be increased with more training

samples.

"""

print(__doc__)



import numpy as np

import matplotlib.pyplot as plt

from sklearn.naive_bayes import GaussianNB

from sklearn.svm import SVC

from sklearn.datasets import load_digits

from sklearn.model_selection import learning_curve

from sklearn.model_selection import ShuffleSplit





def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,

                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):

    """

    Generate a simple plot of the test and training learning curve.



    Parameters

    ----------

    estimator : object type that implements the "fit" and "predict" methods

        An object of that type which is cloned for each validation.



    title : string

        Title for the chart.



    X : array-like, shape (n_samples, n_features)

        Training vector, where n_samples is the number of samples and

        n_features is the number of features.



    y : array-like, shape (n_samples) or (n_samples, n_features), optional

        Target relative to X for classification or regression;

        None for unsupervised learning.



    ylim : tuple, shape (ymin, ymax), optional

        Defines minimum and maximum yvalues plotted.



    cv : int, cross-validation generator or an iterable, optional

        Determines the cross-validation splitting strategy.

        Possible inputs for cv are:

          - None, to use the default 3-fold cross-validation,

          - integer, to specify the number of folds.

          - An object to be used as a cross-validation generator.

          - An iterable yielding train/test splits.



        For integer/None inputs, if ``y`` is binary or multiclass,

        :class:`StratifiedKFold` used. If the estimator is not a classifier

        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.



        Refer :ref:`User Guide <cross_validation>` for the various

        cross-validators that can be used here.



    n_jobs : integer, optional

        Number of jobs to run in parallel (default 1).

    """

    plt.figure()

    plt.title(title)

    if ylim is not None:

        plt.ylim(*ylim)

    plt.xlabel("Training examples")

    plt.ylabel("Score")

    train_sizes, train_scores, test_scores = learning_curve(

        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)

    train_scores_mean = np.mean(train_scores, axis=1)

    train_scores_std = np.std(train_scores, axis=1)

    test_scores_mean = np.mean(test_scores, axis=1)

    test_scores_std = np.std(test_scores, axis=1)

    plt.grid()



    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,

                     train_scores_mean + train_scores_std, alpha=0.1,

                     color="r")

    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,

                     test_scores_mean + test_scores_std, alpha=0.1, color="g")

    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",

             label="Training score")

    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",

             label="Cross-validation score")



    plt.legend(loc="best")

    return plt





digits = load_digits()

X, y = digits.data, digits.target





title = "Learning Curves (Naive Bayes)"

#Cross validation with 100 iterations to get smoother mean test and train

#score curves, each time with 20% data randomly selected as a validation set.

cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)



estimator = GaussianNB()

plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=4)



title = "Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"

#SVC is more expensive so we do a lower number of CV iterations:

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

estimator = SVC(gamma=0.001)

plot_learning_curve(estimator, title, X, y, (0.7, 1.01), cv=cv, n_jobs=4)



plt.show()

In [ ]:
"""

========================

Plotting Learning Curves

========================



On the left side the learning curve of a naive Bayes classifier is shown for

the digits dataset. Note that the training score and the cross-validation score

are both not very good at the end. However, the shape of the curve can be found

in more complex datasets very often: the training score is very high at the

beginning and decreases and the cross-validation score is very low at the

beginning and increases. On the right side we see the learning curve of an SVM

with RBF kernel. We can see clearly that the training score is still around

the maximum and the validation score could be increased with more training

samples.

"""

print(__doc__)



import numpy as np

import matplotlib.pyplot as plt

from sklearn.naive_bayes import GaussianNB

from sklearn.svm import SVC

from sklearn.datasets import load_digits

from sklearn.model_selection import learning_curve

from sklearn.model_selection import ShuffleSplit





def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,

                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):

    """

    Generate a simple plot of the test and training learning curve.



    Parameters

    ----------

    estimator : object type that implements the "fit" and "predict" methods

        An object of that type which is cloned for each validation.



    title : string

        Title for the chart.



    X : array-like, shape (n_samples, n_features)

        Training vector, where n_samples is the number of samples and

        n_features is the number of features.



    y : array-like, shape (n_samples) or (n_samples, n_features), optional

        Target relative to X for classification or regression;

        None for unsupervised learning.



    ylim : tuple, shape (ymin, ymax), optional

        Defines minimum and maximum yvalues plotted.



    cv : int, cross-validation generator or an iterable, optional

        Determines the cross-validation splitting strategy.

        Possible inputs for cv are:

          - None, to use the default 3-fold cross-validation,

          - integer, to specify the number of folds.

          - An object to be used as a cross-validation generator.

          - An iterable yielding train/test splits.



        For integer/None inputs, if ``y`` is binary or multiclass,

        :class:`StratifiedKFold` used. If the estimator is not a classifier

        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.



        Refer :ref:`User Guide <cross_validation>` for the various

        cross-validators that can be used here.



    n_jobs : integer, optional

        Number of jobs to run in parallel (default 1).

    """

    plt.figure()

    plt.title(title)

    if ylim is not None:

        plt.ylim(*ylim)

    plt.xlabel("Training examples")

    plt.ylabel("Score")

    train_sizes, train_scores, test_scores = learning_curve(

        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)

    train_scores_mean = np.mean(train_scores, axis=1)

    train_scores_std = np.std(train_scores, axis=1)

    test_scores_mean = np.mean(test_scores, axis=1)

    test_scores_std = np.std(test_scores, axis=1)

    plt.grid()



    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,

                     train_scores_mean + train_scores_std, alpha=0.1,

                     color="r")

    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,

                     test_scores_mean + test_scores_std, alpha=0.1, color="g")

    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",

             label="Training score")

    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",

             label="Cross-validation score")



    plt.legend(loc="best")

    return plt





digits = load_digits()

X, y = digits.data, digits.target





title = "Learning Curves (Naive Bayes)"

# Cross validation with 100 iterations to get smoother mean test and train

# score curves, each time with 20% data randomly selected as a validation set.

cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)



estimator = GaussianNB()

plot_learning_curve(estimator, title, X, y, ylim=(0.7, 1.01), cv=cv, n_jobs=4)



title = "Learning Curves (SVM, RBF kernel, $\gamma=0.001$)"

# SVC is more expensive so we do a lower number of CV iterations:

cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

estimator = SVC(gamma=0.001)

plot_learning_curve(estimator, title, X, y, (0.7, 1.01), cv=cv, n_jobs=4)



plt.show()

In [ ]:
## Adapted from http://scikit-learn.org/stable/auto_examples/plot_learning_curve.html
import matplotlib.pyplot as plt
from sklearn.learning_curve import learning_curve

# assume classifier and training data is prepared...

train_sizes, train_scores, test_scores = learning_curve(
        forest, X, y, cv=10, n_jobs=-1, train_sizes=np.linspace(.1, 1., 10), verbose=0)

train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.figure()
plt.title("RandomForestClassifier")
plt.legend(loc="best")
plt.xlabel("Training examples")
plt.ylabel("Score")
plt.ylim((0.6, 1.01))
plt.gca().invert_yaxis()
plt.grid()

# Plot the average training and test score lines at each training set size
plt.plot(train_sizes, train_scores_mean, 'o-', color="b", label="Training score")
plt.plot(train_sizes, test_scores_mean, 'o-', color="r", label="Test score")

# Plot the std deviation as a transparent range at each training set size
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, 
                 alpha=0.1, color="b")
plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, 
                 alpha=0.1, color="r")

# Draw the plot and reset the y-axis
plt.draw()
plt.show()
plt.gca().invert_yaxis()

<a id='#Hyperparameters'></a>

# Hyper-parameter optimisation

[[ go back to the top ]](#Table-of-contents)
Phrased as a search problem, you can use different search strategies to find a good and robust parameter or set of parameters for an algorithm on a given problem.<br>
Two simple and easy search strategies are grid search and random search. Scikit-learn provides these two methods for algorithm parameter tuning and examples of each are provided below.<br>
#### Grid Search Parameter Tuning
Grid search is an approach to parameter tuning that will methodically build and evaluate a model for each combination of algorithm parameters specified in a grid. This is a one-dimensional grid search.<br>
<br>
from sklearn.linear_model import Ridge<br>
from sklearn.grid_search import GridSearchCV<br>
#prepare a range of alpha values to test<br>
alphas = numpy.array([1,0.1,0.01,0.001,0.0001,0])<br>
#create and fit a ridge regression model, testing each alpha <br>
model = Ridge()<br>
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas))<br>
grid.fit(X,Y)<br>
print(grid)<br>
#summarize the results of the grid search<br>
print(grid.best_score_)<br>
print(grid.best_estimator_.alpha)<br>
##### Random Search Parameter Tuning
Random search is an approach to parameter tuning that will sample algorithm parameters from a random distribution (i.e. uniform) for a fixed number of iterations. A model is constructed and evaluated for each combination of parameters chosen.<br>
<br>
from scipy.stats import uniform as sp_rand<br>
from sklearn.linear_model import Ridge<br>
from sklearn.grid_search import RandomizedSearchCV<br>
#prepare a uniform distribution to sample for the alpha parameter<br>
param_grid = {'alpha': sp_rand()}<br>
#create and fit a ridge regression model, testing random alpha values<br>
model = Ridge()<br>
rsearch = RandomizedSearchCV(estimator=model, param_distributions=param_grid, n_iter=100) <br>
rsearch.fit(X,Y) <br>
print(rsearch) <br>
#summarize the results of the random parameter search <br>
print(rsearch.best_score_) <br>
print(rsearch.best_estimator_.alpha) <br>

In [ ]:
from sklearn.grid_search import GridSearchCV

decision_tree_classifier = DecisionTreeClassifier()

parameter_grid = {'max_depth': [1, 2, 3, 4, 5],
                  'max_features': [1, 2, 3, 4]}

cross_validation = StratifiedKFold(all_classes, n_folds=10)

grid_search = GridSearchCV(decision_tree_classifier,
                           param_grid=parameter_grid,
                           cv=cross_validation)

grid_search.fit(all_inputs, all_classes)
print('Best score: {}'.format(grid_search.best_score_))
print('Best parameters: {}'.format(grid_search.best_params_))

In [ ]:
grid_visualization = []

for grid_pair in grid_search.grid_scores_:
    grid_visualization.append(grid_pair.mean_validation_score)
    
grid_visualization = np.array(grid_visualization)
grid_visualization.shape = (5, 4)
sb.heatmap(grid_visualization, cmap='Blues')
plt.xticks(np.arange(4) + 0.5, grid_search.param_grid['max_features'])
plt.yticks(np.arange(5) + 0.5, grid_search.param_grid['max_depth'][::-1])
plt.xlabel('max_features')
plt.ylabel('max_depth')

In [ ]:
# Utility function to report optimal parameters
# (adapted from http://scikit-learn.org/stable/auto_examples/randomized_search.html)
def report(grid_scores, n_top=5):
    params = None
    top_scores = sorted(grid_scores, key=itemgetter(1), reverse=True)[:n_top]
    for i, score in enumerate(top_scores):
        print("Parameters with rank: {0}".format(i + 1))
        print("Mean validation score: {0:.4f} (std: {1:.4f})".format(
              score.mean_validation_score, np.std(score.cv_validation_scores)))
        print("Parameters: {0}".format(score.parameters))
        print("")
        
        if params == None:
            params = score.parameters
    
    return params

# The most common value for the max number of features to look at in each split is sqrt(# of features)
sqrtfeat = np.sqrt(X.shape[1]) 

# Simple grid test (162 combinations)
grid_test1 = { "n_estimators"      : [1000, 2500, 5000],
               "criterion"         : ["gini", "entropy"],
               "max_features"      : [sqrtfeat-1, sqrtfeat, sqrtfeat+1],
               "max_depth"         : [5, 10, 25],
               "min_samples_split" : [2, 5, 10] }

# Large randomized test using max_depth to control tree size (5000 possible combinations)
random_test1 = { "n_estimators"      : np.rint(np.linspace(X.shape[0]*2, X.shape[0]*4, 5)).astype(int),
                 "criterion"         : ["gini", "entropy"],
                 "max_features"      : np.rint(np.linspace(sqrtfeat/2, sqrtfeat*2, 5)).astype(int),
                 "max_depth"         : np.rint(np.linspace(1, X.shape[1]/2, 10),
                 "min_samples_split" : np.rint(np.linspace(2, X.shape[0]/50, 10)).astype(int) }

# Large randomized test using min_samples_leaf and max_leaf_nodes to control tree size (50k combinations)
random_test2 = { "n_estimators"      : np.rint(np.linspace(X.shape[0]*2, X.shape[0]*4, 5)).astype(int),
                 "criterion"         : ["gini", "entropy"],
                 "max_features"      : np.rint(np.linspace(sqrtfeat/2, sqrtfeat*2, 5)).astype(int),
                 "min_samples_split" : np.rint(np.linspace(2, X.shape[0]/50, 10)).astype(int),
                 "min_samples_leaf"  : np.rint(np.linspace(1, X.shape[0]/200, 10)).astype(int), 
                 "max_leaf_nodes"    : np.rint(np.linspace(10, X.shape[0]/50, 10)).astype(int) }

forest = RandomForestClassifier(oob_score=True)

print "Hyperparameter optimization using GridSearchCV..."
grid_search = GridSearchCV(forest, grid_test1, n_jobs=-1, cv=10)
grid_search.fit(X, y)
best_params_from_grid_search = scorereport.report(grid_search.grid_scores_)

print "Hyperparameter optimization using RandomizedSearchCV with max_depth parameter..."
grid_search = RandomizedSearchCV(forest, random_test1, n_jobs=-1, cv=10, n_iter=100)
grid_search.fit(X, y)
best_params_from_rand_search1 = scorereport.report(grid_search.grid_scores_)

print "...and using RandomizedSearchCV with min_samples_leaf + max_leaf_nodes parameters..."
grid_search = RandomizedSearchCV(forest, random_test2, n_jobs=-1, cv=10, n_iter=500)
grid_search.fit(X, y)
best_params_from_rand_search2 = scorereport.report(grid_search.grid_scores_)

<a id='#Pipeline'></a>

# A single Pipeline

[[ go back to the top ]](#Table-of-contents)


In [ ]:
%matplotlib inline
import pandas as pd
import seaborn as sb
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import cross_val_score

# We can jump directly to working with the clean data because we saved our cleaned data set
iris_data_clean = pd.read_csv('iris-data-clean.csv')

# Testing our data: Our analysis will stop here if any of these assertions are wrong

# We know that we should only have three classes
assert len(iris_data_clean['class'].unique()) == 3

# We know that sepal lengths for 'Iris-versicolor' should never be below 2.5 cm
assert iris_data_clean.loc[iris_data_clean['class'] == 'Iris-versicolor', 'sepal_length_cm'].min() >= 2.5

# We know that our data set should have no missing measurements
assert len(iris_data_clean.loc[(iris_data_clean['sepal_length_cm'].isnull()) |
                               (iris_data_clean['sepal_width_cm'].isnull()) |
                               (iris_data_clean['petal_length_cm'].isnull()) |
                               (iris_data_clean['petal_width_cm'].isnull())]) == 0

all_inputs = iris_data_clean[['sepal_length_cm', 'sepal_width_cm',
                             'petal_length_cm', 'petal_width_cm']].values

all_classes = iris_data_clean['class'].values

# This is the classifier that came out of Grid Search
random_forest_classifier = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                                max_depth=None, max_features=3, max_leaf_nodes=None,
                                min_samples_leaf=1, min_samples_split=2,
                                min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=1,
                                oob_score=False, random_state=None, verbose=0, warm_start=True)

# All that's left to do now is plot the cross-validation scores
rf_classifier_scores = cross_val_score(random_forest_classifier, all_inputs, all_classes, cv=10)
sb.boxplot(rf_classifier_scores)
sb.stripplot(rf_classifier_scores, jitter=True, color='white')

# ...and show some of the predictions from the classifier
(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(all_inputs, all_classes, train_size=0.75)

random_forest_classifier.fit(training_inputs, training_classes)

for input_features, prediction, actual in zip(testing_inputs[:10],
                                              random_forest_classifier.predict(testing_inputs[:10]),
                                              testing_classes[:10]):
    print('{}\t-->\t{}\t(Actual: {})'.format(input_features, prediction, actual))

<a id='#Pickle'></a>

# Finalize the model

[[ go back to the top ]](#Table-of-contents)
##### Finalize Your Model with pickle
Pickle is the standard way of serializing objects in Python.
You can use the pickle operation to serialize your machine learning algorithms and save the serialized format to a file.
Later you can load this file to deserialize your model and use it to make new predictions.
<br>
#Running the example saves the model to finalized_model.sav in your local working directory<br>
import pickle<br>
test_size = 0.33<br>
seed = 7<br>
X_train, X_test, Y_train, Y_test = cross_validation.train_test_split(X, Y, test_size=test_size, random_state=seed)<br>
#Fit the model on 33%<br>
model = LogisticRegression()<br>
model.fit(X_train, Y_train)<br>
#save the model to disk<br>
filename = 'finalized_model.sav'<br>
pickle.dump(model, open(filename, 'wb'))<br>
<br>
#load the model from disk<br>
loaded_model = pickle.load(open(filename, 'rb'))<br>
result = loaded_model.score(X_test, Y_test)<br>
print(result)<br>

<a id='#time'></a>

# If I had more time ....

[[ go back to the top ]](#Table-of-contents)

<a id='#Acknowledgement'></a>

# Acknowledgements

[[ go back to the top ]](#Table-of-contents)
<br>
<br>
http://www.markhneedham.com/blog/2015/02/15/pythonscikit-learn-calculating-tfidf-on-how-i-met-your-mother-transcripts/<br>
http://machinelearningmastery.com<br>
Notebook by [Randal S. Olson](http://www.randalolson.com/)<br>
http://www.ultravioletanalytics.com<br>
https://www.analyticsvidhya.com<br>
https://sebastianraschka.com/<br>